In [1]:
import pandas as pd
import numpy as np
import os
import operator
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.float_format = '{:20,.2f}'.format
%matplotlib inline
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv(r'C:\Users\ezchave\Desktop\New_data_rankprct\KM_anon_cell_incidents_all.csv')
#df2 = pd.read_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\band700_range20181101-20190430_on20190805_clean.csv')
df.date = pd.to_datetime(df.date)
cell_name_map = pd.read_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\KM_anon_cell_name_key.csv')
df.date = pd.to_datetime(df.date)

In [3]:
df = pd.merge(df,cell_name_map,how='inner',on='cell_name_anon')
df.drop('cell_name_anon',axis=1,inplace=True)
df.rename(columns = {'cell_name':'cell_name_anon'},inplace=True)

In [4]:
df.shape

(2458082, 31)

In [6]:
df.head()

,date,ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS,CALL_SETUP_DELAY_VOLTE_IMS,DROP_CC_408_VOLTE_IMS,DROP_CC_481_VOLTE_IMS,DROP_CC_503_VOLTE_IMS,DROP_CC_OTHER_VOLTE_IMS,DROP_SRVCC_TERMINATE_VOLTE_IMS,GARBLING_VOLTE_VOICE,IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE,MUTING_VOLTE_VOICE,SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS,sum,call_count,est_count_handover,sum_drop_cc_503_481,GARBLING_VOLTE_VOICE_norm,MUTING_VOLTE_VOICE_norm,DROP_CC_481_VOLTE_IMS_norm,CALL_SETUP_DELAY_VOLTE_IMS_norm,SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS_norm,ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS_norm,DROP_CC_503_VOLTE_IMS_norm,DROP_CC_OTHER_VOLTE_IMS_norm,DROP_CC_408_VOLTE_IMS_norm,IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE_norm,DROP_SRVCC_TERMINATE_VOLTE_IMS_norm,sum_norm,band,start_date,cell_name_anon
0,2018-08-01,1,1,0,13,1,0,0,269,0,34,1,320,442,85,0.03,0.61,0.08,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.72,band1900_1,2018-08-01,BBK04000A11
1,2019-05-24,5,3,0,0,0,0,0,106,0,21,1,136,463,169,0.00,0.23,0.05,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.29,band1900_1,2019-05-24,BBK04000A11
2,2018-08-05,0,0,0,3,0,0,0,162,1,11,2,179,320,51,0.01,0.51,0.03,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.56,band1900_1,2018-08-05,BBK04000A11
3,2019-05-23,8,1,0,3,2,0,0,114,0,17,0,145,514,151,0.01,0.22,0.03,0.01,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.28,band1900_1,2019-05-23,BBK04000A11
4,2018-12-02,14,1,0,7,1,0,0,166,0,14,1,204,406,109,0.02,0.41,0.03,0.02,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.50,band1900_1,2018-12-02,BBK04000A11


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2458082 entries, 0 to 2458081
Data columns (total 31 columns):
date                                                     datetime64[ns]
ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS                   int64
CALL_SETUP_DELAY_VOLTE_IMS                               int64
DROP_CC_408_VOLTE_IMS                                    int64
DROP_CC_481_VOLTE_IMS                                    int64
DROP_CC_503_VOLTE_IMS                                    int64
DROP_CC_OTHER_VOLTE_IMS                                  int64
DROP_SRVCC_TERMINATE_VOLTE_IMS                           int64
GARBLING_VOLTE_VOICE                                     int64
IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE         int64
MUTING_VOLTE_VOICE                                       int64
SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS                    int64
sum                                                      int64
call_count                                               in

In [8]:
# separate band 2100 cells
df_band2100=df[df.band=="band2100"]

In [9]:
# reset row indexes
df_band2100=df_band2100.reset_index(drop=True)

In [10]:
df_band2100.shape

(1094129, 31)

In [11]:
df_band2100['band'].value_counts()

band2100    1094129
Name: band, dtype: int64

In [12]:
# how many unique cells are there of band 2100
df_band2100['cell_name_anon'].nunique()

4055

In [13]:
# get unique cells 
df_band2100['cell_name_anon'].unique()

array(['LBK04000A11', 'LBK04000A21', 'LBK04000A31', ..., 'LNYY8323A11',
       'LNYY8323A21', 'LNYY8323A31'], dtype=object)

In [14]:
# transfer the start_date to date format 
df_band2100.start_date = pd.to_datetime(df_band2100.start_date) 

In [15]:
# and sort the data by cell_name_anon and date 
df_band2100.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094129 entries, 0 to 1094128
Data columns (total 31 columns):
date                                                     1094129 non-null datetime64[ns]
ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS                   1094129 non-null int64
CALL_SETUP_DELAY_VOLTE_IMS                               1094129 non-null int64
DROP_CC_408_VOLTE_IMS                                    1094129 non-null int64
DROP_CC_481_VOLTE_IMS                                    1094129 non-null int64
DROP_CC_503_VOLTE_IMS                                    1094129 non-null int64
DROP_CC_OTHER_VOLTE_IMS                                  1094129 non-null int64
DROP_SRVCC_TERMINATE_VOLTE_IMS                           1094129 non-null int64
GARBLING_VOLTE_VOICE                                     1094129 non-null int64
IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE         1094129 non-null int64
MUTING_VOLTE_VOICE                                       1094129 non-null int6

In [16]:
df_band2100.sort_values(by=['cell_name_anon', 'date'],inplace=True)

In [17]:
print(df_band2100['date'].max()); print(df_band2100['date'].min())

2019-05-26 00:00:00
2018-08-01 00:00:00


In [18]:
# Print the list of dates from teh column "date" so that we know what dates are missing from starting to end 
date_list=df_band2100['date'].unique()
print(date_list)

['2018-08-01T00:00:00.000000000' '2018-08-02T00:00:00.000000000'
 '2018-08-03T00:00:00.000000000' '2018-08-04T00:00:00.000000000'
 '2018-08-05T00:00:00.000000000' '2018-08-06T00:00:00.000000000'
 '2018-08-07T00:00:00.000000000' '2018-08-08T00:00:00.000000000'
 '2018-08-09T00:00:00.000000000' '2018-08-10T00:00:00.000000000'
 '2018-08-11T00:00:00.000000000' '2018-08-12T00:00:00.000000000'
 '2018-08-13T00:00:00.000000000' '2018-08-14T00:00:00.000000000'
 '2018-08-15T00:00:00.000000000' '2018-08-16T00:00:00.000000000'
 '2018-08-17T00:00:00.000000000' '2018-08-18T00:00:00.000000000'
 '2018-08-19T00:00:00.000000000' '2018-08-20T00:00:00.000000000'
 '2018-08-21T00:00:00.000000000' '2018-08-22T00:00:00.000000000'
 '2018-08-23T00:00:00.000000000' '2018-08-24T00:00:00.000000000'
 '2018-08-25T00:00:00.000000000' '2018-08-26T00:00:00.000000000'
 '2018-08-27T00:00:00.000000000' '2018-08-28T00:00:00.000000000'
 '2018-08-29T00:00:00.000000000' '2018-08-30T00:00:00.000000000'
 '2018-08-31T00:00:00.000

# We have data from 01-08-2018 till 26-05-2019, 

In [19]:
df_band2100.head()

,date,ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS,CALL_SETUP_DELAY_VOLTE_IMS,DROP_CC_408_VOLTE_IMS,DROP_CC_481_VOLTE_IMS,DROP_CC_503_VOLTE_IMS,DROP_CC_OTHER_VOLTE_IMS,DROP_SRVCC_TERMINATE_VOLTE_IMS,GARBLING_VOLTE_VOICE,IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE,MUTING_VOLTE_VOICE,SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS,sum,call_count,est_count_handover,sum_drop_cc_503_481,GARBLING_VOLTE_VOICE_norm,MUTING_VOLTE_VOICE_norm,DROP_CC_481_VOLTE_IMS_norm,CALL_SETUP_DELAY_VOLTE_IMS_norm,SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS_norm,ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS_norm,DROP_CC_503_VOLTE_IMS_norm,DROP_CC_OTHER_VOLTE_IMS_norm,DROP_CC_408_VOLTE_IMS_norm,IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE_norm,DROP_SRVCC_TERMINATE_VOLTE_IMS_norm,sum_norm,band,start_date,cell_name_anon
35,2018-08-01,2,2,4,3,4,0,0,226,0,41,2,284,1321,261,0.01,0.17,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.21,band2100,2018-08-01,LBK04000A11
107,2018-08-02,4,7,0,8,0,2,0,230,0,33,5,289,1227,231,0.01,0.19,0.03,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.24,band2100,2018-08-02,LBK04000A11
123,2018-08-03,7,17,0,9,1,0,0,245,0,46,5,330,1191,285,0.01,0.21,0.04,0.01,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.28,band2100,2018-08-03,LBK04000A11
36,2018-08-04,8,2,0,4,2,0,0,123,0,12,0,151,594,151,0.01,0.21,0.02,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.25,band2100,2018-08-04,LBK04000A11
16,2018-08-05,4,6,0,3,2,1,0,149,0,27,0,192,675,148,0.01,0.22,0.04,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.28,band2100,2018-08-05,LBK04000A11


In [20]:
#test_cell_5073=df_band2100[df_band2100.cell_name_anon=="cell-5073"]
test_cell_5073=df_band2100[df_band2100.cell_name_anon=="cell-5073"]

In [21]:
test_cell_5073.shape

(0, 31)

In [22]:
df_band2100.columns

Index(['date', 'ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS',
       'CALL_SETUP_DELAY_VOLTE_IMS', 'DROP_CC_408_VOLTE_IMS',
       'DROP_CC_481_VOLTE_IMS', 'DROP_CC_503_VOLTE_IMS',
       'DROP_CC_OTHER_VOLTE_IMS', 'DROP_SRVCC_TERMINATE_VOLTE_IMS',
       'GARBLING_VOLTE_VOICE',
       'IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE',
       'MUTING_VOLTE_VOICE', 'SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS', 'sum',
       'call_count', 'est_count_handover', 'sum_drop_cc_503_481',
       'GARBLING_VOLTE_VOICE_norm', 'MUTING_VOLTE_VOICE_norm',
       'DROP_CC_481_VOLTE_IMS_norm', 'CALL_SETUP_DELAY_VOLTE_IMS_norm',
       'SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS_norm',
       'ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS_norm',
       'DROP_CC_503_VOLTE_IMS_norm', 'DROP_CC_OTHER_VOLTE_IMS_norm',
       'DROP_CC_408_VOLTE_IMS_norm',
       'IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE_norm',
       'DROP_SRVCC_TERMINATE_VOLTE_IMS_norm', 'sum_norm', 'band', 'start_date',
       'cell_name_anon'],
     

In [23]:
grp_dict1= {'ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS' :'sum',
        'CALL_SETUP_DELAY_VOLTE_IMS': 'sum',
        'DROP_CC_408_VOLTE_IMS' : 'sum',
       #'DROP_CC_481_VOLTE_IMS': 'sum',
       #'DROP_CC_503_VOLTE_IMS': 'sum',
       'DROP_CC_OTHER_VOLTE_IMS': 'sum',
       'GARBLING_VOLTE_VOICE': 'sum',
       'IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE': 'sum',
       'MUTING_VOLTE_VOICE': 'sum', 
      'SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS':  'sum',
       'sum_drop_call_503_481': 'sum',
       'call_count': 'sum'} 

In [24]:
grp_dict1

{'ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS': 'sum',
 'CALL_SETUP_DELAY_VOLTE_IMS': 'sum',
 'DROP_CC_408_VOLTE_IMS': 'sum',
 'DROP_CC_OTHER_VOLTE_IMS': 'sum',
 'GARBLING_VOLTE_VOICE': 'sum',
 'IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE': 'sum',
 'MUTING_VOLTE_VOICE': 'sum',
 'SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS': 'sum',
 'sum_drop_call_503_481': 'sum',
 'call_count': 'sum'}

In [25]:
features1 = ['ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS', 
            'CALL_SETUP_DELAY_VOLTE_IMS',
       'DROP_CC_408_VOLTE_IMS', 'DROP_CC_OTHER_VOLTE_IMS',
       'GARBLING_VOLTE_VOICE',
       'IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE',
       'MUTING_VOLTE_VOICE', 'SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS','hard_drop_call_503_481'
       ]

In [26]:

rank_dates=['2018-08-08',
       '2018-08-09', '2018-08-10', '2018-08-11', '2018-08-12',
       '2018-08-13', '2018-08-14', '2018-08-15', '2018-08-16',
       '2018-08-17', '2018-08-18', '2018-08-19', '2018-08-20',
       '2018-08-21', '2018-08-22', '2018-08-23', '2018-08-24',
       '2018-08-25', '2018-08-26', '2018-08-27', '2018-08-28',
       '2018-08-29', '2018-08-30', '2018-08-31', '2018-09-01',
       '2018-09-02', '2018-09-03', '2018-09-04', '2018-09-05',
       '2018-09-06', '2018-09-07', '2018-09-09', '2018-09-10',
       '2018-09-11', '2018-09-12', '2018-09-13', '2018-09-14',
       '2018-09-15', '2018-09-16', '2018-09-17', '2018-09-18',
       '2018-09-19', '2018-09-20', '2018-09-21', '2018-09-22',
       '2018-09-23', '2018-09-24', '2018-09-25', '2018-09-26',
       '2018-09-27', '2018-09-28', '2018-09-29', '2018-09-30',
       '2018-10-01', '2018-10-02', '2018-10-03', '2018-10-04',
       '2018-10-05', '2018-10-06', '2018-10-07', '2018-10-08',
       '2018-10-09', '2018-10-10', '2018-10-11', '2018-10-12',
       '2018-10-13', '2018-10-14', '2018-10-15', '2018-10-16',
       '2018-10-17', '2018-10-18', '2018-10-19', '2018-10-20',
       '2018-10-21', '2018-10-22', '2018-10-23', '2018-10-24',
       '2018-10-25', '2018-10-26', '2018-10-27', '2018-10-28',
       '2018-11-01', '2018-11-02', '2018-11-03', '2018-11-04',
       '2018-11-05', '2018-11-06', '2018-11-07', '2018-11-08',
       '2018-11-09', '2018-11-10', '2018-11-11', '2018-11-12',
       '2018-11-13', '2018-11-14', '2018-11-15', '2018-11-16',
       '2018-11-17', '2018-11-18', '2018-11-19', '2018-11-20',
       '2018-11-21', '2018-11-22', '2018-11-23', '2018-11-24',
       '2018-11-25', '2018-11-26', '2018-11-27', '2018-11-29',
       '2018-11-30', '2018-12-01', '2018-12-02', '2018-12-03',
       '2018-12-04', '2018-12-05', '2018-12-06', '2018-12-07',
       '2018-12-08', '2018-12-09', '2018-12-10', '2018-12-11',
       '2018-12-12', '2018-12-13', '2018-12-14', '2018-12-15',
       '2018-12-16', '2018-12-17', '2018-12-18', '2018-12-19',
       '2018-12-20', '2018-12-21', '2018-12-23', '2018-12-24',
       '2018-12-25', '2018-12-26', '2018-12-27', '2018-12-28',
       '2018-12-29', '2018-12-30', '2018-12-31', '2019-01-01',
       '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05',
       '2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09',
       '2019-01-10', '2019-01-11', '2019-01-12', '2019-01-13',
       '2019-01-14', '2019-01-15', '2019-01-16', '2019-01-23',
       '2019-01-24', '2019-01-25', '2019-01-27', '2019-01-28',
       '2019-01-29', '2019-01-30', '2019-01-31', '2019-02-01',
       '2019-02-02', '2019-02-03', '2019-02-04', '2019-02-05',
       '2019-02-06', '2019-02-07', '2019-02-08', '2019-02-11',
       '2019-02-12', '2019-02-13', '2019-02-14', '2019-02-15',
       '2019-02-16', '2019-02-18', '2019-02-19'
    ]

In [27]:
df.head()

,date,ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS,CALL_SETUP_DELAY_VOLTE_IMS,DROP_CC_408_VOLTE_IMS,DROP_CC_481_VOLTE_IMS,DROP_CC_503_VOLTE_IMS,DROP_CC_OTHER_VOLTE_IMS,DROP_SRVCC_TERMINATE_VOLTE_IMS,GARBLING_VOLTE_VOICE,IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE,MUTING_VOLTE_VOICE,SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS,sum,call_count,est_count_handover,sum_drop_cc_503_481,GARBLING_VOLTE_VOICE_norm,MUTING_VOLTE_VOICE_norm,DROP_CC_481_VOLTE_IMS_norm,CALL_SETUP_DELAY_VOLTE_IMS_norm,SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS_norm,ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS_norm,DROP_CC_503_VOLTE_IMS_norm,DROP_CC_OTHER_VOLTE_IMS_norm,DROP_CC_408_VOLTE_IMS_norm,IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE_norm,DROP_SRVCC_TERMINATE_VOLTE_IMS_norm,sum_norm,band,start_date,cell_name_anon
0,2018-08-01,1,1,0,13,1,0,0,269,0,34,1,320,442,85,0.03,0.61,0.08,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.72,band1900_1,2018-08-01,BBK04000A11
1,2019-05-24,5,3,0,0,0,0,0,106,0,21,1,136,463,169,0.00,0.23,0.05,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.29,band1900_1,2019-05-24,BBK04000A11
2,2018-08-05,0,0,0,3,0,0,0,162,1,11,2,179,320,51,0.01,0.51,0.03,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.56,band1900_1,2018-08-05,BBK04000A11
3,2019-05-23,8,1,0,3,2,0,0,114,0,17,0,145,514,151,0.01,0.22,0.03,0.01,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.28,band1900_1,2019-05-23,BBK04000A11
4,2018-12-02,14,1,0,7,1,0,0,166,0,14,1,204,406,109,0.02,0.41,0.03,0.02,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.50,band1900_1,2018-12-02,BBK04000A11


In [28]:
def Rolling_data_PCA_Percentile(df,rank_dates, nr_days, features):
    # output path
    Output_path='C:\My_work\MXE\CM_Config\output'
    dates = df.date.unique()
    dates = sorted(dates)
    for i in range(len(dates)):
        #test_dates[i] = date(test_dates[i])
        dates[i] = pd.to_datetime(dates[i])
        dates[i] = dates[i].strftime('%Y-%m-%d')
    # Coverting date column to datetime datatype
    #df.date = pd.to_datetime(df.date)
    dates1 = df.date
    # creating Blank data frames for storing the output of the iterations 
    rolledup_data=pd.DataFrame(columns=["ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS","CALL_SETUP_DELAY_VOLTE_IMS",
                                        "DROP_CC_408_VOLTE_IMS","DROP_CC_OTHER_VOLTE_IMS","GARBLING_VOLTE_VOICE",
                                        "IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE","MUTING_VOLTE_VOICE",
                                        "SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS","hard_drop_call_503_481",
                                        "cell_name_anon","band","date"])

    final_df_band2100 = pd.DataFrame(columns=["cell_name_anon","band","score","date","rank","percentile"])
    final_df_band700 = pd.DataFrame(columns=["cell_name_anon","band","score","date","rank","percentile"])
   # final_df_band1900_1 = pd.DataFrame(columns=["cell_name_anon","band","score","date","rank","percentile"])
        
    for rank_date in rank_dates:
        # coverting rank_date string to date 
        this_date = rank_date
        #rank_date = dt.date(*map(int, rank_date.split('-')))
        print(rank_date)
        # Preparing the window for 7 days
        
        #start_date = rank_date - dt.timedelta(1)
        #end_date = start_date - dt.timedelta(nr_days)
        #rank_date = '2019-03-01T00:00:00.000000000'
        #this_date = np.datetime64(this_date)
        i = dates.index(this_date)
        start_date = dates[i-1]
        end_date = dates[i-nr_days]
        #print(df['date'].head(2))
        dff = df[(df['date'] < rank_date) & (df['date'] >= end_date)]
        # CHANGE MADE HERE
        dff['hard_drop_call_503_481']= dff['DROP_CC_481_VOLTE_IMS']+ dff['DROP_CC_503_VOLTE_IMS']
         # CHNAGE MADE HERE
        # grouping by cell_name_anon , band
        grp_dict1= {'ACCESS_FAILURE_CC_503.1.223._VOLTE_IMS' :'sum',
            'CALL_SETUP_DELAY_VOLTE_IMS': 'sum',
            'DROP_CC_408_VOLTE_IMS' : 'sum',
           'DROP_CC_OTHER_VOLTE_IMS': 'sum',
           'GARBLING_VOLTE_VOICE': 'sum',
           'IMMEDIATE_VOLUNTARY_DROP_DUE_TO_MEDIA_STOP_VOLTE': 'sum',
           'MUTING_VOLTE_VOICE': 'sum', 
           'SOFT_DROP_DUE_TO_MEDIA_STOP_VOLTE_IMS':  'sum',
           'hard_drop_call_503_481': 'sum',
           'call_count': 'sum'} 
    
        df1 = dff.groupby(['cell_name_anon','band']).agg(grp_dict1 , as_index = False)
    
        df1.reset_index(inplace=True)
    
        df2= df1.iloc[:,2:].div(df1.call_count, axis=0)
        df3 = pd.concat([df2.iloc[:,0:9], df1[['cell_name_anon','band']]], axis = 1)
        
        df3['date']=rank_date
         # append all iterations
        rolledup_data=rolledup_data.append(df3,ignore_index = True)
    
        ##########################################
        # ----------------Band 2100---------- ####
        ##########################################
        # Separating out the features
        df_band2100 = df3[df3.band == 'band2100']
        df_band2100.reset_index(inplace = True)
        x = df_band2100.loc[:, features1].values
        # Standardizing the features
        x = StandardScaler().fit_transform(x)
        # doing PCA
        pca = PCA()
        principalComponents = pca.fit_transform(x)
        # converting to dataframe
        principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3',
                          'principal component 4','principal component 5', 'principal component 6',
                         'principal component 7','principal component 8', 'principal component 9'
                          ])
        # selecting where explained variance >= 1
        exp_variance = pca.explained_variance_
        # fetching indexes
        idx = np.where(exp_variance >= 1)[0]
        #print(idx) ; print(type(idx[0]))
        #fetching multipliers
        exp_ratio = pca.explained_variance_ratio_[idx]
        # slicing Principal components and adding cell_name_anon & band
        cols=['cell_name_anon', 'band']
        _df_band2100 =  pd.concat([principalDf.iloc[:,idx], df_band2100[cols]], axis = 1)
        _df_band2100['score'] = np.sum(_df_band2100.iloc[:,idx] * exp_ratio , axis = 1)
        _df_band2100['date'] = rank_date
        _df_band2100.sort_values(by=['score'],ascending=True , inplace=True)
        _df_band2100.reset_index(inplace= True)
        _df_band2100['rank'] = _df_band2100.index.values+1
        _df_band2100 = _df_band2100[['cell_name_anon','band','score','date','rank']]
        
        _df_band2100['percentile'] = _df_band2100.score.rank(pct = True)
        _df_band2100.percentile = 1 - _df_band2100.percentile 
        _df_band2100.percentile = _df_band2100.percentile * 100
              
        # append all iterations
        final_df_band2100=_df_band2100.append(final_df_band2100,ignore_index = True)
        #print('success')
        print(final_df_band2100.head(2))
               
        ##########################################
        # ----------------Band 700---------- ####
        ##########################################
        # Separating out the features
        df_band700 = df3[df3.band == 'band700']
        df_band700.reset_index(inplace = True)
        x = df_band700.loc[:, features1].values
        # Standardizing the features
        x = StandardScaler().fit_transform(x)
        # doing PCA
        pca = PCA()
        principalComponents = pca.fit_transform(x)
        # converting to dataframe
        principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3',
                          'principal component 4','principal component 5', 'principal component 6',
                         'principal component 7','principal component 8', 'principal component 9'
                          ])
        # selecting where explained variance >= 1
        exp_variance = pca.explained_variance_
        # fetching indexes
        idx = np.where(exp_variance >= 1)[0]
        #print(idx) ; print(type(idx[0]))
        #fetching multipliers
        exp_ratio = pca.explained_variance_ratio_[idx]
        # slicing Principal components and adding cell_name_anon & band
        cols=['cell_name_anon', 'band']
        _df_band700 =  pd.concat([principalDf.iloc[:,idx], df_band700[cols]], axis = 1)
        _df_band700['score'] = np.sum(_df_band700.iloc[:,idx] * exp_ratio , axis = 1)
        _df_band700['date'] = rank_date
        _df_band700.sort_values(by=['score'],ascending=True , inplace=True)
        _df_band700.reset_index(inplace= True)
        _df_band700['rank'] = _df_band700.index.values+1
        _df_band700 = _df_band700[['cell_name_anon','band','score','date','rank']]
        _df_band700['percentile'] = _df_band700.score.rank(pct = True)
        _df_band700.percentile = 1 - _df_band700.percentile 
        _df_band700.percentile = _df_band700.percentile * 100
        
        # append all iterations
        final_df_band700=_df_band700.append(final_df_band700,ignore_index = True)
        #print('success')
        print(final_df_band700.head(2))
        
    return final_df_band2100,final_df_band700, rolledup_data  

In [29]:
def apply_PCA_on_period(start_date):
    # k for mean +- k*std
    #start_date = '2019-05-11'
    dates = df.date.unique()
    dates = sorted(dates)
    start_date = np.datetime64(start_date)
    i = dates.index(start_date)
    test_dates = dates[i-29:i+1]
    build_dates = dates[i-119:i-29]
    for i in range(len(test_dates)):
        #test_dates[i] = date(test_dates[i])
        test_dates[i] = pd.to_datetime(test_dates[i])
        test_dates[i] = test_dates[i].strftime('%Y-%m-%d')
    for i in range(len(build_dates)):
        #test_dates[i] = date(test_dates[i])
        build_dates[i] = pd.to_datetime(build_dates[i])
        build_dates[i] = build_dates[i].strftime('%Y-%m-%d')
    print(test_dates)
    #make the df for first 13 weeks
    Consolidated_df_2100_build,Consolidated_df_700_build, Consolidated_data_all_build = Rolling_data_PCA_Percentile(df, build_dates, 7, features1)
    #make the df for the one month of test
    Consolidated_df_2100_test,Consolidated_df_700_test, Consolidated_data_all_test = Rolling_data_PCA_Percentile(df,test_dates, 7, features1)
    print('PCA done \n')
    
    return Consolidated_df_700_build,Consolidated_df_700_test

In [30]:
def calculate_performance(Consolidated_df_2100_build,Consolidated_df_2100_test,k):
    cell_rank = Consolidated_df_2100_build[['cell_name_anon','score','rank']]
    cell_rank['rank'] = cell_rank['rank'].astype(int)
    cell_rank_mean = cell_rank.groupby('cell_name_anon').mean()
    cell_rank_mean.reset_index(inplace = True)
    cell_rank_std = cell_rank.groupby('cell_name_anon').std()
    cell_rank_std.reset_index(inplace = True)
    #cell_rank_std.head()
    cell_rank_mean['score_std'] = cell_rank_std.score
    cell_rank_mean['rank_std'] = cell_rank_std['rank']
    cell_rank_mean.rename(columns = {'score' : 'score_mean_90days','rank':'rank_mean'},inplace = True)
    cell_rank_mean.head()
    cell_score_mean = cell_rank_mean.copy()
    del(cell_rank_mean)
    cell_score_mean['mu+sigma'] = cell_score_mean['score_mean_90days'] + (k*cell_score_mean['score_std'])
    cell_score_mean['mu-sigma'] = cell_score_mean['score_mean_90days'] - (k*cell_score_mean['score_std'])
    
    print('cell ranking for build period done \n')
    # getting the mean of the best days of cell and standard deviation for all the days in the build period

    Consolidated_df_2100_build = pd.merge(Consolidated_df_2100_build,cell_score_mean[['cell_name_anon','score_std','score_mean_90days','mu+sigma','mu-sigma']],how='inner',on='cell_name_anon')
    Consolidated_df_2100_build['gt_mu+sigma'] = Consolidated_df_2100_build['score'] > Consolidated_df_2100_build['mu+sigma']
    Consolidated_df_2100_build['lt_mu-sigma'] = Consolidated_df_2100_build['score'] < Consolidated_df_2100_build['mu+sigma']
    Consolidated_df_2100_build = Consolidated_df_2100_build.sort_values(by=['cell_name_anon','date'])
    
    cell_best_days = Consolidated_df_2100_build[Consolidated_df_2100_build['lt_mu-sigma']==True]
    x = cell_best_days[['cell_name_anon','score']].groupby('cell_name_anon').mean()
    
    x.reset_index(inplace=True)
    y = Consolidated_df_2100_build[['cell_name_anon','score']].groupby('cell_name_anon').std()
    y.reset_index(inplace=True)
    #Consolidated_df_2100_build = pd.merge(Consolidated_df_2100_build,x[['cell_name_anon','score']],on='cell_name_anon',how='inner')

    del(cell_best_days)
    
    print('cell mean and standard deviation calculation done \n')

    cell_mean_std = pd.merge(x,y,on='cell_name_anon',how='inner')
    cell_mean_std.rename(columns ={'score_x':'score_mean_good_days','score_y':'score_std_90days'},inplace=True)
    Consolidated_df_2100_build = pd.merge(Consolidated_df_2100_build,cell_mean_std[['cell_name_anon','score_mean_good_days']],on='cell_name_anon',how='inner')
    Consolidated_df_2100_test= pd.merge(Consolidated_df_2100_test,cell_mean_std,how='inner',on='cell_name_anon')
    Consolidated_df_2100_test.sort_values(by=['cell_name_anon','date'],inplace=True)
    Consolidated_df_2100_test.reset_index(inplace=True,drop=True)
    Consolidated_df_2100_test = pd.merge(Consolidated_df_2100_test,cell_score_mean[['cell_name_anon','score_mean_90days']],how='inner',on='cell_name_anon')
    #print(cell_score_mean.head())
    del(cell_score_mean)
    Consolidated_df_2100_test['mu+'+str(k)+'sigma'] = Consolidated_df_2100_test['score_mean_good_days'] + (k*Consolidated_df_2100_test['score_std_90days'])
    Consolidated_df_2100_test['mu-'+str(k)+'sigma'] = Consolidated_df_2100_test['score_mean_good_days'] - (k*Consolidated_df_2100_test['score_std_90days'])
    Consolidated_df_2100_test['gt_mu+'+str(k)+'sigma'] = Consolidated_df_2100_test['score'] > Consolidated_df_2100_test['mu+'+str(k)+'sigma']
    Consolidated_df_2100_test['lt_mu-'+str(k)+'sigma'] = Consolidated_df_2100_test['score'] < Consolidated_df_2100_test['mu-'+str(k)+'sigma']
    #Consolidated_df_2100_test.head()
    Consolidated_df_2100_test.rename(columns={'score':'mean_score_90days'},inplace=True)
    print('cell mean +- sigma calculation done for test period')
    print(Consolidated_df_2100_test.head())
    return Consolidated_df_2100_test,Consolidated_df_2100_build

In [31]:
sorted(df.date.unique())

[numpy.datetime64('2018-08-01T00:00:00.000000000'),
 numpy.datetime64('2018-08-02T00:00:00.000000000'),
 numpy.datetime64('2018-08-03T00:00:00.000000000'),
 numpy.datetime64('2018-08-04T00:00:00.000000000'),
 numpy.datetime64('2018-08-05T00:00:00.000000000'),
 numpy.datetime64('2018-08-06T00:00:00.000000000'),
 numpy.datetime64('2018-08-07T00:00:00.000000000'),
 numpy.datetime64('2018-08-08T00:00:00.000000000'),
 numpy.datetime64('2018-08-09T00:00:00.000000000'),
 numpy.datetime64('2018-08-10T00:00:00.000000000'),
 numpy.datetime64('2018-08-11T00:00:00.000000000'),
 numpy.datetime64('2018-08-12T00:00:00.000000000'),
 numpy.datetime64('2018-08-13T00:00:00.000000000'),
 numpy.datetime64('2018-08-14T00:00:00.000000000'),
 numpy.datetime64('2018-08-15T00:00:00.000000000'),
 numpy.datetime64('2018-08-16T00:00:00.000000000'),
 numpy.datetime64('2018-08-17T00:00:00.000000000'),
 numpy.datetime64('2018-08-18T00:00:00.000000000'),
 numpy.datetime64('2018-08-19T00:00:00.000000000'),
 numpy.datet

In [32]:
x=129
test_k2_ = {}
build_k2_ = {}
n=1
test_k2 = pd.DataFrame()
dates = sorted(df.date.unique())
max_date = len(dates)-1
while x <= max_date:
    print(pd.to_datetime(dates[x]).strftime('%Y-%m-%d'))
    pca_build,pca_test = apply_PCA_on_period(pd.to_datetime(dates[x]).strftime('%Y-%m-%d'))
    test_k2_[n],build_k2_[n] = calculate_performance(pca_build,pca_test,2)
    x += 30
    n+=1
for x in range(1,len(test_k2_)):
    test_k2 = test_k2.append(test_k2_[x])
#del(test_k2_,build_k2_)  
 
test_k2.date = pd.to_datetime(test_k2.date)

2018-12-13
['2018-11-13', '2018-11-14', '2018-11-15', '2018-11-16', '2018-11-17', '2018-11-18', '2018-11-19', '2018-11-20', '2018-11-21', '2018-11-22', '2018-11-23', '2018-11-24', '2018-11-25', '2018-11-26', '2018-11-27', '2018-11-29', '2018-11-30', '2018-12-01', '2018-12-02', '2018-12-03', '2018-12-04', '2018-12-05', '2018-12-06', '2018-12-07', '2018-12-08', '2018-12-09', '2018-12-10', '2018-12-11', '2018-12-12', '2018-12-13']
2018-08-11


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


  cell_name_anon      band                score        date rank  \
0    LBQ04564A11  band2100                -0.81  2018-08-11    1   
1    LNYB0801A41  band2100                -0.79  2018-08-11    2   

            percentile  
0                99.98  
1                99.95  
  cell_name_anon     band                score        date rank  \
0    DBQ04254A21  band700                -1.09  2018-08-11    1   
1    DBQ04254A31  band700                -1.05  2018-08-11    2   

            percentile  
0                99.97  
1                99.93  
2018-08-12


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


  cell_name_anon      band                score        date rank  \
0    LBQ04564A11  band2100                -0.84  2018-08-12    1   
1    LNYB0801A41  band2100                -0.74  2018-08-12    2   

            percentile  
0                99.98  
1                99.95  
  cell_name_anon     band                score        date rank  \
0    DBQ04254A21  band700                -1.11  2018-08-12    1   
1    DBQ06083B31  band700                -1.09  2018-08-12    2   

            percentile  
0                99.97  
1                99.93  
2018-08-13


...

  cell_name_anon      band                score        date rank  \
0    LNYB0803A41  band2100                -1.60  2019-05-02    1   
1    LNY02043B21  band2100                -1.46  2019-05-02    2   

            percentile  
0                99.98  
1                99.95  
  cell_name_anon     band                score        date rank  \
0    DNY01038A11  band700                -0.68  2019-05-02    1   
1    DBQ04813C21  band700                -0.67  2019-05-02    2   

            percentile  
0                99.97  
1                99.93  
2019-05-03


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


  cell_name_anon      band                score        date rank  \
0    LNYB0803A41  band2100                -1.87  2019-05-03    1   
1    LNY02043B21  band2100                -1.48  2019-05-03    2   

            percentile  
0                99.98  
1                99.95  
  cell_name_anon     band                score        date rank  \
0    DNY01038A11  band700                -0.72  2019-05-03    1   
1    DBX03036B21  band700                -0.70  2019-05-03    2   

            percentile  
0                99.97  
1                99.93  
PCA done 



C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


cell ranking for build period done 

cell mean and standard deviation calculation done 

cell mean +- sigma calculation done for test period
  cell_name_anon     band    mean_score_90days        date  rank  \
0    DBK04000A11  band700                 0.67  2019-04-04  2560   
1    DBK04000A11  band700                 0.67  2019-04-05  2562   
2    DBK04000A11  band700                 0.57  2019-04-06  2515   
3    DBK04000A11  band700                 0.46  2019-04-07  2423   
4    DBK04000A11  band700                 0.53  2019-04-08  2490   

            percentile  score_mean_good_days     score_std_90days  \
0                11.02                  0.42                 0.20   
1                10.95                  0.42                 0.20   
2                12.58                  0.42                 0.20   
3                15.75                  0.42                 0.20   
4                13.42                  0.42                 0.20   

     score_mean_90days            m

In [33]:

#pca_build,pca_test = apply_PCA_on_period('2018-12-31') # last date must be after 120 days of data 
#test_k05,build_k05 = calculate_performance(pca_build,pca_test,0.5)
#test_k1,build_k1 = calculate_performance(pca_build,pca_test,1)
#test_k15,build_k15 = calculate_performance(pca_build,pca_test,1.5)
#test_k2,build_k2 = calculate_performance(pca_build,pca_test,2)
#test_k25,build_k25 = calculate_performance(pca_build,pca_test,2.5)
#test_k3,build_k3 = calculate_performance(pca_build,pca_test,3)
#test_k2_full = test_k2.copy()
#test_k2 = test_k2[test_k2.cell_name_anon == 'cell-5073']
#test_k2 = test_k2_full.copy()
pca_build,pca_test = apply_PCA_on_period(pd.to_datetime('2019-05-04'))
test_k2_[6],build_k2_[6] = calculate_performance(pca_build,pca_test,2)
test_k2=test_k2.append(test_k2_[6])
test_k2.date = pd.to_datetime(test_k2.date)
test_k2.date.unique()

['2019-04-05', '2019-04-06', '2019-04-07', '2019-04-08', '2019-04-09', '2019-04-10', '2019-04-11', '2019-04-12', '2019-04-13', '2019-04-14', '2019-04-15', '2019-04-16', '2019-04-17', '2019-04-18', '2019-04-19', '2019-04-20', '2019-04-21', '2019-04-22', '2019-04-23', '2019-04-24', '2019-04-25', '2019-04-26', '2019-04-27', '2019-04-28', '2019-04-29', '2019-04-30', '2019-05-01', '2019-05-02', '2019-05-03', '2019-05-04']
2018-12-15


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


  cell_name_anon      band                score        date rank  \
0    LNY01526I81  band2100                -1.28  2018-12-15    1   
1    LNY01335F31  band2100                -1.04  2018-12-15    2   

            percentile  
0                99.98  
1                99.95  
  cell_name_anon     band                score        date rank  \
0    DNY01038A11  band700                -0.91  2018-12-15    1   
1    DBKM4099A21  band700                -0.91  2018-12-15    2   

            percentile  
0                99.97  
1                99.93  
2018-12-16


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


  cell_name_anon      band                score        date rank  \
0    LNY01526I81  band2100                -1.66  2018-12-16    1   
1    LNYY8120A21  band2100                -1.12  2018-12-16    2   

            percentile  
0                99.98  
1                99.95  
  cell_name_anon     band                score        date rank  \
0    DBKM4099A21  band700                -0.93  2018-12-16    1   
1    DNY01038A11  band700                -0.92  2018-12-16    2   

            percentile  
0                99.97  
1                99.93  
2018-12-17


...

C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


  cell_name_anon      band                score        date rank  \
0    LNYB0803A41  band2100                -1.78  2019-05-04    1   
1    LNY02043B21  band2100                -1.36  2019-05-04    2   

            percentile  
0                99.98  
1                99.95  
  cell_name_anon     band                score        date rank  \
0    DNY01073B11  band700                -0.72  2019-05-04    1   
1    DBX03036B21  band700                -0.70  2019-05-04    2   

            percentile  
0                99.97  
1                99.93  
PCA done 



C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


cell ranking for build period done 

cell mean and standard deviation calculation done 

cell mean +- sigma calculation done for test period
  cell_name_anon     band    mean_score_90days        date  rank  \
0    DBK04000A11  band700                 0.67  2019-04-05  2562   
1    DBK04000A11  band700                 0.57  2019-04-06  2515   
2    DBK04000A11  band700                 0.46  2019-04-07  2423   
3    DBK04000A11  band700                 0.53  2019-04-08  2490   
4    DBK04000A11  band700                 0.56  2019-04-09  2483   

            percentile  score_mean_good_days     score_std_90days  \
0                10.95                  0.42                 0.20   
1                12.58                  0.42                 0.20   
2                15.75                  0.42                 0.20   
3                13.42                  0.42                 0.20   
4                13.66                  0.42                 0.20   

     score_mean_90days            m

array(['2018-11-13T00:00:00.000000000', '2018-11-14T00:00:00.000000000',
       '2018-11-15T00:00:00.000000000', '2018-11-16T00:00:00.000000000',
       '2018-11-17T00:00:00.000000000', '2018-11-18T00:00:00.000000000',
       '2018-11-19T00:00:00.000000000', '2018-11-20T00:00:00.000000000',
       '2018-11-21T00:00:00.000000000', '2018-11-22T00:00:00.000000000',
       '2018-11-23T00:00:00.000000000', '2018-11-24T00:00:00.000000000',
       '2018-11-25T00:00:00.000000000', '2018-11-26T00:00:00.000000000',
       '2018-11-27T00:00:00.000000000', '2018-11-29T00:00:00.000000000',
       '2018-11-30T00:00:00.000000000', '2018-12-01T00:00:00.000000000',
       '2018-12-02T00:00:00.000000000', '2018-12-03T00:00:00.000000000',
       '2018-12-04T00:00:00.000000000', '2018-12-05T00:00:00.000000000',
       '2018-12-06T00:00:00.000000000', '2018-12-07T00:00:00.000000000',
       '2018-12-08T00:00:00.000000000', '2018-12-09T00:00:00.000000000',
       '2018-12-10T00:00:00.000000000', '2018-12-11

In [34]:
test_k2.date.unique()

array(['2018-11-13T00:00:00.000000000', '2018-11-14T00:00:00.000000000',
       '2018-11-15T00:00:00.000000000', '2018-11-16T00:00:00.000000000',
       '2018-11-17T00:00:00.000000000', '2018-11-18T00:00:00.000000000',
       '2018-11-19T00:00:00.000000000', '2018-11-20T00:00:00.000000000',
       '2018-11-21T00:00:00.000000000', '2018-11-22T00:00:00.000000000',
       '2018-11-23T00:00:00.000000000', '2018-11-24T00:00:00.000000000',
       '2018-11-25T00:00:00.000000000', '2018-11-26T00:00:00.000000000',
       '2018-11-27T00:00:00.000000000', '2018-11-29T00:00:00.000000000',
       '2018-11-30T00:00:00.000000000', '2018-12-01T00:00:00.000000000',
       '2018-12-02T00:00:00.000000000', '2018-12-03T00:00:00.000000000',
       '2018-12-04T00:00:00.000000000', '2018-12-05T00:00:00.000000000',
       '2018-12-06T00:00:00.000000000', '2018-12-07T00:00:00.000000000',
       '2018-12-08T00:00:00.000000000', '2018-12-09T00:00:00.000000000',
       '2018-12-10T00:00:00.000000000', '2018-12-11

In [35]:
def remove_n_days (test_k,k,n):
    test_k.reset_index(inplace=True,drop=True)
    index = list(test_k.index)
    count = 0
    to_stay = []
    for i in range(n):
        index.pop(len(index)-1)
    print('test_k before taking only ',str(n),' straight days of deviation has ', test_k.shape[0],' rows')
    
    for i in index :
        val = []
        same_cell = True
        if(test_k.iloc[i]['cell_name_anon'] != test_k.loc[i+n]['cell_name_anon']):
            same_cell = False 
        for kk in range(n):
            val.append(test_k.loc[i+kk,'gt_mu+'+str(k)+'sigma'])
        if(all(val) and same_cell):
            to_stay.append(i-1)
            for kk in range(n):
                to_stay.append(i+kk)
            count += 1
    to_stay = list(set(to_stay))
    test_k = test_k.iloc[to_stay]
    print('test_k after taking only ',str(n),' straight days of deviation has ',test_k.shape[0],' rows')
    test_k.reset_index(inplace=True,drop=True)
    print('found such deviations : ',count)
    return test_k


In [36]:
def make_changes_df(test_k,k):

    #making the changes dataFrame for the cells on the days having low performance
    test_k.date = pd.to_datetime(test_k.date)
    x = test_k[test_k['gt_mu+'+str(k) +'sigma']==True][['cell_name_anon','date']]
    x.date = x.date - dt.timedelta(days=1)
    changes_df = test_k[test_k['gt_mu+'+str(k) +'sigma']==True].append(pd.merge(test_k,x,how='inner',on=['cell_name_anon','date']))
    changes_df.sort_values(by=['cell_name_anon','date'],inplace=True)
    changes_df.drop_duplicates(inplace = True)
    changes_df.reset_index(inplace=True,drop=True)
    index = list(changes_df.index)
    true_to_go = []
    index.pop()
    for i in index:
        if(changes_df.loc[i,'gt_mu+'+str(k) +'sigma'] == True) & (changes_df.loc[i+1,'gt_mu+'+str(k) +'sigma'] == True):
            true_to_go.append(i+1)
    changes_df.drop(true_to_go,axis=0,inplace=True)
    changes_df.sort_values(by=['cell_name_anon','date'],inplace=True)
    print(changes_df.shape)
    print(changes_df[changes_df['gt_mu+'+str(k) +'sigma']==True].shape)
    del(true_to_go,x,i,index)
    return changes_df

In [37]:
#changes_df = make_changes_df(test_k2.copy(),2)
features = ['est_count_handover',
'call_count',
'vsDataEUtranCellFDD__crsGain',
'feature_UplinkCarrierAggregation',
'feature_UlTrigInterFreqMob',
'vsDataEUtranCellFDD__pdcchTargetBlerVolte',
'feature_QuadAntUlPerfPkg',
'feature_QuadAntDlPerfPkg4x4',
'feature_QuadAntDlPerfPkg',
'feature_VoLTEOptimizedCA',
'dotw',
'vsDataEUtranCellFDD__ulSrsEnable',
'feature_UlSpectrumAnalyzer',
'vsDataENodeBFunction__dlBbCapacityNet',
'feature_CrossDUCarrierAggregation',
'feature_FddTddSameENodeB',
'vsDataEUtranCellFDD__puschPwrOffset64qam',
'vsDataENodeBFunction__timePhaseMaxDeviationOtdoa',
'vsDataENodeBFunction__ulBbCapacityNet',
'vsDataEUtranCellFDD__cellCapMinCellSubCap',
'feature_Support12Cells',
'qciProfile__tReorderingDl',
'vsDataENodeBFunction__caAwareMfbiIntraCellHo',
'feature_AdvancedDiffForResourceFairSch',
'vsDataEUtranCellFDD__outOfCoverageSrTimerPeriodicity',
'vsDataENodeBFunction__enabledUlTrigMeas',
'feature_CarrierAggregationFddTdd',
'feature_FourDlCarrierAggregation',
'feature_MixedModeRadioLte',
'vsDataEUtranCellFDD__additionalPlmnList_mcc',
'vsDataENodeBFunction__releaseInactiveUesInactTime',
'feature_UlInterferenceReporting',
'vsDataEUtranCellFDD__pZeroNominalPusch',
'vsDataEUtranCellFDD__eDrxAllowed',
'vsDataEUtranCellFDD__threshServingLow',
'vsDataENodeBFunction__mfbiSupport',
'feature_SrvccToUtran',
'feature_DynamicScellSelection',
'vsDataENodeBFunction__inactivitySupervisionTimerNb',
'feature_MultiFreqBand',
'vsDataEUtranCellFDD__cellDownlinkCaCapacity',
'feature_ThreeDlCarrierAggregation',
'vsDataEUtranCellFDD__lastSchedLinkAdaptEnabled',
'feature_RelInactiveUeAtHighLoadHo',
'feature_PucchOverdimensioning',
'feature_PdcchPowerBoost',
'feature_UeThroughputAwareIflb',
'qciProfile__inactivityTimerOffset',
'feature_SgwRelocationAtX2Handover','cell_name','date'
]

In [38]:
#changes_df = make_changes_df(test_k2.copy(),2)
features1 = ['est_count_handover',
'call_count',
'vsDataEUtranCellFDD__crsGain',
'feature_UplinkCarrierAggregation',
'feature_UlTrigInterFreqMob',
'vsDataEUtranCellFDD__pdcchTargetBlerVolte',
'feature_QuadAntUlPerfPkg',
'feature_QuadAntDlPerfPkg4x4',
'feature_QuadAntDlPerfPkg',
'feature_VoLTEOptimizedCA',
'dotw',
'vsDataEUtranCellFDD__ulSrsEnable',
'feature_UlSpectrumAnalyzer',
'vsDataENodeBFunction__dlBbCapacityNet',
'feature_CrossDUCarrierAggregation',
'feature_FddTddSameENodeB',
'vsDataEUtranCellFDD__puschPwrOffset64qam',
'vsDataENodeBFunction__timePhaseMaxDeviationOtdoa',
'vsDataENodeBFunction__ulBbCapacityNet',
'vsDataEUtranCellFDD__cellCapMinCellSubCap',
'feature_Support12Cells',
'qciProfile__tReorderingDl',
'vsDataENodeBFunction__caAwareMfbiIntraCellHo',
'feature_AdvancedDiffForResourceFairSch',
'vsDataEUtranCellFDD__outOfCoverageSrTimerPeriodicity',
'vsDataENodeBFunction__enabledUlTrigMeas',
'feature_CarrierAggregationFddTdd',
'feature_FourDlCarrierAggregation',
'feature_MixedModeRadioLte',
'vsDataEUtranCellFDD__additionalPlmnList_mcc',
'vsDataENodeBFunction__releaseInactiveUesInactTime',
'feature_UlInterferenceReporting',
'vsDataEUtranCellFDD__pZeroNominalPusch',
'vsDataEUtranCellFDD__eDrxAllowed',
'vsDataEUtranCellFDD__threshServingLow',
'vsDataENodeBFunction__mfbiSupport',
'feature_SrvccToUtran',
'feature_DynamicScellSelection',
'vsDataENodeBFunction__inactivitySupervisionTimerNb',
'feature_MultiFreqBand',
'vsDataEUtranCellFDD__cellDownlinkCaCapacity',
'feature_ThreeDlCarrierAggregation',
'vsDataEUtranCellFDD__lastSchedLinkAdaptEnabled',
'feature_RelInactiveUeAtHighLoadHo',
'feature_PucchOverdimensioning',
'feature_PdcchPowerBoost',
'feature_UeThroughputAwareIflb',
'qciProfile__inactivityTimerOffset',
'feature_SgwRelocationAtX2Handover','cell_name_anon','date'
]

In [39]:
def get_deviation_df(test_k,cm_data,k):
    #this is the function to get data only for the day of deviation and the day before
    # and the inceident data of the same days
    #test_k = test dataframe generated before
    #cm_data is the dataframe of the cm counters
    #k is the value of k in the test dataframe sent above
    #test_k=test_k15.copy()
    #k=1.5
    test_k.date = pd.to_datetime(test_k.date)
    x = test_k[test_k['gt_mu+'+str(k)+'sigma']==True][['cell_name_anon','date']]
    y = x.copy()
    x.date = x.date - dt.timedelta(days=1)
    y=y.append(x)
    y.reset_index(inplace=True)
    changes_df = test_k[test_k['gt_mu+'+str(k)+'sigma']==True].append(pd.merge(test_k,x,how='inner',on=['cell_name_anon','date']))
    changes_df.sort_values(by=['cell_name_anon','date'],inplace=True)
    changes_df.drop_duplicates(inplace = True)
    changes_df.reset_index(inplace=True,drop=True)
    index = list(changes_df.index)
    true_to_go = []
    index.pop()
    for i in index:
        if(changes_df.loc[i,'gt_mu+'+str(k)+'sigma'] == True) & (changes_df.loc[i+1,'gt_mu+'+str(k)+'sigma'] == True):
            true_to_go.append(i+1)
    changes_df.drop(true_to_go,axis=0,inplace=True)
    changes_df.sort_values(by=['cell_name_anon','date'],inplace=True)
    #print(changes_df.shape)
    #print(changes_df[changes_df['gt_mu+'+str(k)+'sigma']==True].shape)
    #del(true_to_go,x,i,index)
    cm_data.date = pd.to_datetime(cm_data.date)
    changes_cm = pd.DataFrame()
    changes_df = pd.merge(changes_df,df,on=['cell_name_anon','date'],how='inner')
    #print(changes_df.shape)
    changes_cm = pd.merge(changes_df[['cell_name_anon','date']],cm_data,on=['cell_name_anon','date'],how='left')
    
    #print('changes_cm shape : ',changes_cm.shape)
    #print(changes_df.shape)
    changes_df.drop(['band_x'],inplace=True,axis=1)
    changes_df.rename(columns={'band_y':'band','call_count_y':'call_count','est_count_handover_y':'est_count_handover'},inplace=True)
    changes_cm['unique'] = changes_cm['cell_name_anon'].astype(str) + changes_cm['date'].astype(str)
    
    to_go = []
    i=0
    while(i<(len(changes_cm.index)-1)):
        date_today,cell_today = changes_cm.loc[i,['date','cell_name_anon']]
        date_tomo,cell_tomo = changes_cm.loc[i+1,['date','cell_name_anon']]
        date_today = date_today + dt.timedelta(days=1)
        if(cell_today == cell_tomo):
            if(date_today!=date_tomo):
                to_go.append((date_today,i,changes_cm.loc[i,'cell_name_anon']))
                i=i+1
            else:
                i=i+2
        else:
            i=i+1
    for i in to_go :
        changes_cm.drop(i[1],axis=0,inplace=True)
    changes_cm.reset_index(inplace=True,drop=True)
    if((changes_cm.shape[0])%2 != 0 ):
        changes_cm.drop(changes_cm.shape[0]-1,axis=0,inplace=True)
    changes_cm.index = changes_cm['unique'] 
    #print('changes_cm shape : ',changes_cm.shape)
    changes_cm.drop(['cell_name_anon','date','unique'],inplace=True,axis=1)
    changes_cm.reset_index(inplace=True)

    #print('changes_cm shape : ',changes_cm.shape)
    #changes_cm.index = changes_cm['unique'].values
    #changes_cm.drop('unique',axis=1,inplace=True)
    #print(changes_cm.head())
    #del(test_k,cm_data,k)
    return changes_cm,changes_df

In [74]:
def get_counter_info(changes_cm):
    # this is the function to get the value of the changes in the cm counters on the day of deveiation and before
    #changes cm is the df having the cm data for the days of deviation and the days before
    changes_cm.fillna('n',inplace=True)
    if(changes_cm.shape[0]%2 != 0):
        changes_cm.drop(changes_cm.shape[0]-1,inplace=True,axis=0)
    for col in changes_cm.columns:
        changes_cm[col]=changes_cm[col].astype(str)
    changes_cm_even = changes_cm.loc[0:len(changes_cm):2,:]
    changes_cm_odd = changes_cm.loc[1:len(changes_cm):2,:]
    changes_cm_even_index = changes_cm_even['unique']
    changes_cm_odd_index = changes_cm_odd['unique']
    changes_cm_odd.index = changes_cm_odd['unique']
    changes_cm_even.index = changes_cm_even['unique']

    print('changes_cm_even.shape : ',changes_cm_even.shape , '\n changes_cm_odd.shape : ' ,changes_cm_odd.shape )
    changes_cm_even.drop('unique',axis=1,inplace=True)
    changes_cm_odd.drop('unique',axis=1,inplace=True)
    changes_cm_odd.reset_index(drop=True,inplace=True)
    changes_cm_even.reset_index(drop=True,inplace=True)
    del(changes_cm)
    compr_mask = [changes_cm_even.values != changes_cm_odd.values]
    changes_cm_even.index = changes_cm_even.index*2 
    changes_cm_even['unique'] = changes_cm_even_index
    changes_cm_even.reset_index(inplace=True,drop=True)
    changes_cm_odd.index = (changes_cm_odd.index*2) + 1
    changes_cm_odd['unique'] = changes_cm_odd_index
    changes_cm_odd.reset_index(inplace=True,drop=True)
    #compr_mask[0:10]
    change_cm_alt = pd.DataFrame()
    change_cm_alt = changes_cm_even +" , " + changes_cm_odd
    del(changes_cm_even,changes_cm_odd)
    change_cm_alt_uniq = change_cm_alt['unique']
    print(compr_mask)
    change_cm_alt = change_cm_alt.drop('unique',axis=1).mul(compr_mask[0][:])
    change_cm_alt['unique'] = change_cm_alt_uniq
    del(change_cm_alt_uniq)
    #change_cm_alt.head(20)
    x=pd.DataFrame(index = range(change_cm_alt.shape[0]))
    for col in change_cm_alt:
        x1=[]
        for y in change_cm_alt.loc[:,col] :
            try:

                if ('n' not in y) & ('' != y) :
                    x1.append(y)
            except:
                print(y)
        l = len(x1)
        x1 += (change_cm_alt.shape[0] - l) * '&'
        #print(col,len(x1),change_cm_alt.shape[0])
        x[col] = (x1)
    del(x1)
    #change_cm_alt is the dataframe having string tuples of the changed counter values
    # x is the data frame where cols are the features and rows are the different values they take
    # in x all columns are filled with str(0a0) i.e. string 0a0 to make all the columns of the same length
    # ignore the 0 values mentioned in the previous step
    return change_cm_alt,x

In [41]:
######### taking only dec month values
#test_k2.date = pd.to_datetime(test_k2.date)
#test_k2 = test_k2[test_k2.date > pd.to_datetime('2018-11-30')] 
#test_k2 = test_k2[test_k2.date < pd.to_datetime('2018-12-30')]
#test_k2.date.unique()

### now lets take only days with n straight days of deviation

In [42]:
test_k2_7 = remove_n_days(test_k2.copy(),2,7)
test_k2_7.drop_duplicates({'cell_name_anon','date'},inplace = True)
print(test_k2_7.shape)

test_k before taking only  7  straight days of deviation has  431521  rows
test_k after taking only  7  straight days of deviation has  18162  rows
found such deviations :  7214
(18162, 13)


In [43]:
test_k2_7.date.unique()

array(['2019-01-15T00:00:00.000000000', '2019-01-16T00:00:00.000000000',
       '2019-01-23T00:00:00.000000000', '2019-01-24T00:00:00.000000000',
       '2019-01-25T00:00:00.000000000', '2019-01-27T00:00:00.000000000',
       '2019-01-28T00:00:00.000000000', '2019-01-29T00:00:00.000000000',
       '2019-01-30T00:00:00.000000000', '2019-01-31T00:00:00.000000000',
       '2019-02-01T00:00:00.000000000', '2019-04-24T00:00:00.000000000',
       '2019-04-25T00:00:00.000000000', '2019-04-26T00:00:00.000000000',
       '2019-04-23T00:00:00.000000000', '2019-04-27T00:00:00.000000000',
       '2019-04-28T00:00:00.000000000', '2019-04-29T00:00:00.000000000',
       '2019-04-30T00:00:00.000000000', '2019-05-01T00:00:00.000000000',
       '2019-05-03T00:00:00.000000000', '2019-05-02T00:00:00.000000000',
       '2018-12-05T00:00:00.000000000', '2018-12-06T00:00:00.000000000',
       '2018-12-07T00:00:00.000000000', '2018-12-08T00:00:00.000000000',
       '2018-12-09T00:00:00.000000000', '2018-12-10

In [123]:
############### getting the new cm_data ---------------------------------------------

#changes_df = make_changes_df(test_k2_7.copy(),2)
#taking the cm data to a table cm_data
chunksize = 10 ** 7
cm_data = pd.DataFrame()

for chunk in pd.read_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\band700_range20181101-20190430_on20190805_clean.csv', chunksize=chunksize):
#for chunk in pd.read_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\band700_dataset_A.csv', chunksize=chunksize):
    #chunk = chunk[['features']]
    chunk = chunk.reindex(columns = features)
    cm_data = cm_data.append(chunk)
    
    
#for chunk in pd.read_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\band700_dataset_B.csv', chunksize=chunksize):
#    #chunk = chunk[['features']]
#    chunk = chunk.reindex(columns = features)
#    cm_data = cm_data.append(chunk)
    
del(chunk,chunksize)



In [126]:
cm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436293 entries, 0 to 436292
Data columns (total 51 columns):
est_count_handover                                      436293 non-null int64
call_count                                              436293 non-null int64
vsDataEUtranCellFDD__crsGain                            0 non-null float64
feature_UplinkCarrierAggregation                        436293 non-null int64
feature_UlTrigInterFreqMob                              436293 non-null int64
vsDataEUtranCellFDD__pdcchTargetBlerVolte               0 non-null float64
feature_QuadAntUlPerfPkg                                436293 non-null int64
feature_QuadAntDlPerfPkg4x4                             436293 non-null int64
feature_QuadAntDlPerfPkg                                436293 non-null int64
feature_VoLTEOptimizedCA                                436293 non-null int64
dotw                                                    436293 non-null int64
vsDataEUtranCellFDD__ulSrsEnable     

In [45]:
df1 = pd.read_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\band700_range20181101-20190430_on20190805_clean.csv')

In [46]:
df1.head()

,RlfProfile..n310,RlfProfile..t310,TimerProfile..tRrcConnectionReconfiguration,TimerProfile..tWaitForRrcConnReest,cell_name,date,feature_AdaptiveRLCPollRetransmission,feature_AdvCellSup,feature_AdvancedDiffForResourceFairSch,feature_AirIfLoadGenerator,feature_Anr,feature_AutoCellCapEstFunction,feature_AutomatedMobilityOptimization,feature_BasebandResourceCellPrio,feature_BnrIntraLteLM,feature_CarrierAggregation,feature_CarrierAggregationAwareIFLB,feature_CarrierAggregationFddTdd,feature_CellSoftLock,feature_ConfigurableScellPrio,feature_CqiPuschSinrPowerInTrace,feature_CrossDUCarrierAggregation,feature_CsfbForLimitedDualRadioUe,feature_CsfbToGeranUtran,feature_DbsAndSabe,feature_DifferentiatedAdmissionControl,feature_DifferentiatedUeHandling,feature_Dl256Qam,feature_DlFss,feature_Drx,feature_DualAntDlPerfPkg,feature_DynamicGbrAdmCtrl,feature_DynamicPucch,feature_DynamicQoSModification,feature_DynamicRABackoff,feature_DynamicScellSelection,feature_EcsCsfb,feature_EnhCellIdInTraces,feature_EnhancedPdcchLa,feature_FddTddSameENodeB,feature_FiveDlCarrierAggregation,feature_FourDlCarrierAggregation,feature_GsmSessionContinuity,feature_HoOscCtrlUE,feature_IFLBActivationThreshold,feature_IdleModeExtendedDrx,feature_InterENBCarrierAggregation,feature_InterFreqSessionContinuity,feature_InterFrequencyLoadBalancing,feature_InterFrequencyLteHandover,feature_Internal1,feature_IntraLteHoDataFwd,feature_Irc,feature_LoadBasedAccessBarring,feature_LsmEcs,feature_MaximumCellRange,feature_MicroSleepTx,feature_MixedModeRadioLte,feature_MmeOverloadControl,feature_MobCtrlAtPoorCov,feature_MultiErabsPerUser,feature_MultiFreqBand,feature_MultiLayerServTriMobility,feature_MultiTargetRrcConnReest,feature_NarrowbandIoTAccess,feature_NbIotLoadDistribution,feature_Nbiot100kmCoverage,feature_NonPlannedPCIRange,feature_OtdoaPrsManagement,feature_OtdoaSupl,feature_PHICHGroupSpreading,feature_Pci,feature_PdcchPowerBoost,feature_Pfs,feature_PmInitiatedUeMeasurements,feature_Prescheduling,feature_PrioritizedSrScheduling,feature_PriorityPaging,feature_PucchOverdimensioning,feature_Pws,feature_QosAwareScheduler,feature_QuadAntDlPerfPkg,feature_QuadAntDlPerfPkg4x4,feature_QuadAntUlPerfPkg,feature_RbsEventStreamer,feature_RelInactiveUeAtHighLoadHo,feature_RlcUm,feature_Rohc,feature_Rps,feature_S1HoDataFwd,feature_SRVCCtoGERAN,feature_SUPPLEMENTARYDLONLYCELL,feature_ServiceSpecificDrx,feature_ServiceSpecificHARQ,feature_ServiceSpecificInactivityTimer,feature_ServiceSpecificLoadMgmt,feature_ServiceTriggeredMobility,feature_SgwRelocationAtX2Handover,feature_SharedNetworkSupport,feature_SrvccToUtran,feature_Support12Cells,feature_Support18Cells,feature_Support24Cells,feature_Support6Cells,feature_TcpOptimization,feature_ThreeDlCarrierAggregation,feature_UeThroughputAwareIflb,feature_Ul64Qam,feature_UlComp,feature_UlDynamicResourceAllocation,feature_UlFss,feature_UlInterferenceReporting,feature_UlSchedContlForOocUes,feature_UlSpectrumAnalyzer,feature_UlTrigInterFreqMob,feature_UplinkCarrierAggregation,feature_VoLTEOptimizedCA,feature_WcdmaHandover,feature_WcdmaSessionContinuity,lat,lon,nodeId,qciProfile..dlMaxHARQTxQci,qciProfile..dlResourceAllocationStrategy,qciProfile..inactivityTimerOffset,qciProfile..measReportConfigParams_a1ThresholdRsrqSecOffset,qciProfile..measReportConfigParams_a2ThresholdRsrqSecOffset,qciProfile..srsAllocationStrategy,qciProfile..tReorderingDl,vsDataENodeBFunction..caAwareMfbiIntraCellHo,vsDataENodeBFunction..dlBbCapacityNet,vsDataENodeBFunction..enabledUlTrigMeas,vsDataENodeBFunction..gtpuErrorIndicationDscp,vsDataENodeBFunction..inactivitySupervisionTimerNb,vsDataENodeBFunction..licCapDistrMethod,vsDataENodeBFunction..mfbiSupport,vsDataENodeBFunction..mfbiSupportPolicy,vsDataENodeBFunction..prioritizeAdditionalBands,vsDataENodeBFunction..releaseInactiveUesInactTime,vsDataENodeBFunction..releaseInactiveUesMpLoadLevel,vsDataENodeBFunction..tRelocOverall,vsDataENodeBFunction..timeAndPhaseSynchAlignment,vsDataENodeBFunction..timeAndPhaseSynchCriti

In [47]:
############## getting the old cm_data ------------------------------

#changes_df = make_changes_df(test_k2_7.copy(),2)
#taking the cm data to a table cm_data
chunksize = 10 ** 7
cm_data_old = pd.DataFrame()

#for chunk in pd.read_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\band700_range20181101-20190430_on20190805_clean.csv', chunksize=chunksize):
for chunk in pd.read_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\band700_dataset_A.csv', chunksize=chunksize):
    #chunk = chunk[['features']]
    chunk = chunk.reindex(columns = features1)
    cm_data_old = cm_data_old.append(chunk)
    
    
for chunk in pd.read_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\band700_dataset_B.csv', chunksize=chunksize):
    #chunk = chunk[['features']]
    chunk = chunk.reindex(columns = features1)
    cm_data_old = cm_data_old.append(chunk)
    
del(chunk,chunksize)



C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (257,258,297,308,309,314,315,325,328,331,343,344,345,347,352,359,372,373,400,402,416,423,445,446,447,456,457,469,475,480,495,501,503,509,511,514,516,517,521,523,524,525,526,527,538,551,552,586,592,612,613,629,630,631,647,648,649,667,671,672,673,685,694,695,703,704,708,709,710) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (252,253,292,303,304,309,310,320,323,326,338,339,340,342,347,354,367,368,395,397,411,418,440,441,442,451,452,464,470,475,490,496,498,506,509,511,512,516,518,519,520,521,522,533,546,547,581,587,607,608,624,625,626,642,643,644,662,666,667,668,680,689,690,698,699,703,704,705) have mixed types. Specify dtype option on import 

In [48]:
cm_data.head()

,est_count_handover,call_count,vsDataEUtranCellFDD__crsGain,feature_UplinkCarrierAggregation,feature_UlTrigInterFreqMob,vsDataEUtranCellFDD__pdcchTargetBlerVolte,feature_QuadAntUlPerfPkg,feature_QuadAntDlPerfPkg4x4,feature_QuadAntDlPerfPkg,feature_VoLTEOptimizedCA,dotw,vsDataEUtranCellFDD__ulSrsEnable,feature_UlSpectrumAnalyzer,vsDataENodeBFunction__dlBbCapacityNet,feature_CrossDUCarrierAggregation,feature_FddTddSameENodeB,vsDataEUtranCellFDD__puschPwrOffset64qam,vsDataENodeBFunction__timePhaseMaxDeviationOtdoa,vsDataENodeBFunction__ulBbCapacityNet,vsDataEUtranCellFDD__cellCapMinCellSubCap,feature_Support12Cells,qciProfile__tReorderingDl,vsDataENodeBFunction__caAwareMfbiIntraCellHo,feature_AdvancedDiffForResourceFairSch,vsDataEUtranCellFDD__outOfCoverageSrTimerPeriodicity,vsDataENodeBFunction__enabledUlTrigMeas,feature_CarrierAggregationFddTdd,feature_FourDlCarrierAggregation,feature_MixedModeRadioLte,vsDataEUtranCellFDD__additionalPlmnList_mcc,vsDataENodeBFunction__releaseInactiveUesInactTime,feature_UlInterferenceReporting,vsDataEUtranCellFDD__pZeroNominalPusch,vsDataEUtranCellFDD__eDrxAllowed,vsDataEUtranCellFDD__threshServingLow,vsDataENodeBFunction__mfbiSupport,feature_SrvccToUtran,feature_DynamicScellSelection,vsDataENodeBFunction__inactivitySupervisionTimerNb,feature_MultiFreqBand,vsDataEUtranCellFDD__cellDownlinkCaCapacity,feature_ThreeDlCarrierAggregation,vsDataEUtranCellFDD__lastSchedLinkAdaptEnabled,feature_RelInactiveUeAtHighLoadHo,feature_PucchOverdimensioning,feature_PdcchPowerBoost,feature_UeThroughputAwareIflb,qciProfile__inactivityTimerOffset,feature_SgwRelocationAtX2Handover,cell_name,date
0,165,1997,nan,0,1,nan,1,1,1,0,3,nan,0,nan,1,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,1,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,0,0,1,0,nan,1,DNY02321B11,2018-11-01
1,134,1033,nan,0,1,nan,1,1,1,0,3,nan,0,nan,1,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,1,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,0,0,1,0,nan,1,DNY02321B21,2018-11-01
2,152,1274,nan,0,1,nan,1,1,1,0,3,nan,0,nan,1,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,1,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,0,0,1,0,nan,1,DNY02321B31,2018-11-01
3,35,117,nan,0,0,nan,0,0,0,0,3,nan,0,nan,0,0,nan,nan,nan,nan,0,nan,nan,1,nan,nan,0,0,0,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,1,1,1,0,nan,1,DNY01037A11,2018-11-01
4,35,587,nan,0,1,nan,0,0,0,0,3,nan,0,nan,1,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,1,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,0,0,1,0,nan,1,DNY02131A11,2018-11-01


### Checking the new cm data

In [49]:
sorted(cm_data.date.unique())

['2018-11-01',
 '2018-11-02',
 '2018-11-03',
 '2018-11-04',
 '2018-11-05',
 '2018-11-06',
 '2018-11-07',
 '2018-11-08',
 '2018-11-10',
 '2018-11-11',
 '2018-11-12',
 '2018-11-13',
 '2018-11-14',
 '2018-11-15',
 '2018-11-16',
 '2018-11-17',
 '2018-11-18',
 '2018-11-19',
 '2018-11-20',
 '2018-11-21',
 '2018-11-22',
 '2018-11-23',
 '2018-11-24',
 '2018-11-25',
 '2018-11-26',
 '2018-11-27',
 '2018-11-29',
 '2018-11-30',
 '2018-12-01',
 '2018-12-02',
 '2018-12-03',
 '2018-12-04',
 '2018-12-05',
 '2018-12-07',
 '2018-12-08',
 '2018-12-09',
 '2018-12-10',
 '2018-12-11',
 '2018-12-12',
 '2018-12-13',
 '2018-12-14',
 '2018-12-15',
 '2018-12-16',
 '2018-12-17',
 '2018-12-18',
 '2018-12-19',
 '2018-12-20',
 '2018-12-21',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29',
 '2018-12-30',
 '2018-12-31',
 '2019-01-01',
 '2019-01-02',
 '2019-01-03',
 '2019-01-04',
 '2019-01-05',
 '2019-01-06',
 '2019-01-07',
 '2019-01-08',
 '2019-01-09',
 '2019-01-

In [50]:
sorted(cm_data_old.date.unique())

['2018-11-01',
 '2018-11-02',
 '2018-11-03',
 '2018-11-04',
 '2018-11-05',
 '2018-11-06',
 '2018-11-07',
 '2018-11-08',
 '2018-11-09',
 '2018-11-10',
 '2018-11-11',
 '2018-11-12',
 '2018-11-13',
 '2018-11-14',
 '2018-11-15',
 '2018-11-16',
 '2018-11-17',
 '2018-11-18',
 '2018-11-19',
 '2018-11-20',
 '2018-11-21',
 '2018-11-22',
 '2018-11-23',
 '2018-11-24',
 '2018-11-25',
 '2018-11-26',
 '2018-11-27',
 '2018-11-29',
 '2018-11-30',
 '2018-12-01',
 '2018-12-02',
 '2018-12-03',
 '2018-12-04',
 '2018-12-05',
 '2018-12-06',
 '2018-12-07',
 '2018-12-08',
 '2018-12-09',
 '2018-12-10',
 '2018-12-11',
 '2018-12-12',
 '2018-12-13',
 '2018-12-14',
 '2018-12-15',
 '2018-12-16',
 '2018-12-17',
 '2018-12-18',
 '2018-12-19',
 '2018-12-20',
 '2018-12-21',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29',
 '2018-12-30',
 '2018-12-31',
 '2019-01-01',
 '2019-01-02',
 '2019-01-03',
 '2019-01-04',
 '2019-01-05',
 '2019-01-06',
 '2019-01-07',
 '2019-01-

In [51]:
cm_data_old.drop_duplicates(inplace = True)

In [52]:
cm_data.drop_duplicates(inplace = True)

In [53]:
cm_data_old.head()

,est_count_handover,call_count,vsDataEUtranCellFDD__crsGain,feature_UplinkCarrierAggregation,feature_UlTrigInterFreqMob,vsDataEUtranCellFDD__pdcchTargetBlerVolte,feature_QuadAntUlPerfPkg,feature_QuadAntDlPerfPkg4x4,feature_QuadAntDlPerfPkg,feature_VoLTEOptimizedCA,dotw,vsDataEUtranCellFDD__ulSrsEnable,feature_UlSpectrumAnalyzer,vsDataENodeBFunction__dlBbCapacityNet,feature_CrossDUCarrierAggregation,feature_FddTddSameENodeB,vsDataEUtranCellFDD__puschPwrOffset64qam,vsDataENodeBFunction__timePhaseMaxDeviationOtdoa,vsDataENodeBFunction__ulBbCapacityNet,vsDataEUtranCellFDD__cellCapMinCellSubCap,feature_Support12Cells,qciProfile__tReorderingDl,vsDataENodeBFunction__caAwareMfbiIntraCellHo,feature_AdvancedDiffForResourceFairSch,vsDataEUtranCellFDD__outOfCoverageSrTimerPeriodicity,vsDataENodeBFunction__enabledUlTrigMeas,feature_CarrierAggregationFddTdd,feature_FourDlCarrierAggregation,feature_MixedModeRadioLte,vsDataEUtranCellFDD__additionalPlmnList_mcc,vsDataENodeBFunction__releaseInactiveUesInactTime,feature_UlInterferenceReporting,vsDataEUtranCellFDD__pZeroNominalPusch,vsDataEUtranCellFDD__eDrxAllowed,vsDataEUtranCellFDD__threshServingLow,vsDataENodeBFunction__mfbiSupport,feature_SrvccToUtran,feature_DynamicScellSelection,vsDataENodeBFunction__inactivitySupervisionTimerNb,feature_MultiFreqBand,vsDataEUtranCellFDD__cellDownlinkCaCapacity,feature_ThreeDlCarrierAggregation,vsDataEUtranCellFDD__lastSchedLinkAdaptEnabled,feature_RelInactiveUeAtHighLoadHo,feature_PucchOverdimensioning,feature_PdcchPowerBoost,feature_UeThroughputAwareIflb,qciProfile__inactivityTimerOffset,feature_SgwRelocationAtX2Handover,cell_name_anon,date
0,215,1205,nan,0,0,nan,1,1,1,0,4,nan,0,nan,0,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,0,nan,nan,0,nan,nan,nan,nan,0,1,nan,0,nan,1,nan,0,1,1,0,nan,1,cell-2196,2018-11-09
1,138,638,nan,0,0,nan,1,1,1,1,6,nan,0,nan,0,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,0,nan,nan,1,nan,nan,nan,nan,0,1,nan,1,nan,1,nan,0,1,1,0,nan,1,cell-2196,2019-04-28
2,240,1214,nan,0,0,nan,1,1,1,0,1,nan,0,nan,0,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,0,nan,nan,1,nan,nan,nan,nan,0,1,nan,1,nan,1,nan,0,1,1,0,nan,1,cell-2196,2019-03-12
3,178,833,nan,0,0,nan,1,1,1,0,0,nan,0,nan,0,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,0,nan,nan,0,nan,nan,nan,nan,0,1,nan,0,nan,1,nan,0,1,1,0,nan,1,cell-2196,2018-11-12
4,267,1118,nan,0,0,nan,1,1,1,0,1,nan,0,nan,0,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,0,nan,nan,0,nan,nan,nan,nan,0,1,nan,0,nan,1,nan,0,1,1,0,nan,1,cell-2196,2018-11-27


In [54]:
##############checking the dates in old but not in new
for x in sorted(cm_data_old.date.unique()):
    if(x not in cm_data.date.unique()):
        print(x)

2018-11-09
2018-12-06
2019-05-01
2019-05-02
2019-05-03
2019-05-04
2019-05-09
2019-05-10
2019-05-11
2019-05-12
2019-05-13
2019-05-14
2019-05-15
2019-05-16
2019-05-17
2019-05-18
2019-05-19
2019-05-20
2019-05-21
2019-05-22
2019-05-23
2019-05-24
2019-05-25


In [55]:
cm_data.head()

,est_count_handover,call_count,vsDataEUtranCellFDD__crsGain,feature_UplinkCarrierAggregation,feature_UlTrigInterFreqMob,vsDataEUtranCellFDD__pdcchTargetBlerVolte,feature_QuadAntUlPerfPkg,feature_QuadAntDlPerfPkg4x4,feature_QuadAntDlPerfPkg,feature_VoLTEOptimizedCA,dotw,vsDataEUtranCellFDD__ulSrsEnable,feature_UlSpectrumAnalyzer,vsDataENodeBFunction__dlBbCapacityNet,feature_CrossDUCarrierAggregation,feature_FddTddSameENodeB,vsDataEUtranCellFDD__puschPwrOffset64qam,vsDataENodeBFunction__timePhaseMaxDeviationOtdoa,vsDataENodeBFunction__ulBbCapacityNet,vsDataEUtranCellFDD__cellCapMinCellSubCap,feature_Support12Cells,qciProfile__tReorderingDl,vsDataENodeBFunction__caAwareMfbiIntraCellHo,feature_AdvancedDiffForResourceFairSch,vsDataEUtranCellFDD__outOfCoverageSrTimerPeriodicity,vsDataENodeBFunction__enabledUlTrigMeas,feature_CarrierAggregationFddTdd,feature_FourDlCarrierAggregation,feature_MixedModeRadioLte,vsDataEUtranCellFDD__additionalPlmnList_mcc,vsDataENodeBFunction__releaseInactiveUesInactTime,feature_UlInterferenceReporting,vsDataEUtranCellFDD__pZeroNominalPusch,vsDataEUtranCellFDD__eDrxAllowed,vsDataEUtranCellFDD__threshServingLow,vsDataENodeBFunction__mfbiSupport,feature_SrvccToUtran,feature_DynamicScellSelection,vsDataENodeBFunction__inactivitySupervisionTimerNb,feature_MultiFreqBand,vsDataEUtranCellFDD__cellDownlinkCaCapacity,feature_ThreeDlCarrierAggregation,vsDataEUtranCellFDD__lastSchedLinkAdaptEnabled,feature_RelInactiveUeAtHighLoadHo,feature_PucchOverdimensioning,feature_PdcchPowerBoost,feature_UeThroughputAwareIflb,qciProfile__inactivityTimerOffset,feature_SgwRelocationAtX2Handover,cell_name,date
0,165,1997,nan,0,1,nan,1,1,1,0,3,nan,0,nan,1,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,1,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,0,0,1,0,nan,1,DNY02321B11,2018-11-01
1,134,1033,nan,0,1,nan,1,1,1,0,3,nan,0,nan,1,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,1,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,0,0,1,0,nan,1,DNY02321B21,2018-11-01
2,152,1274,nan,0,1,nan,1,1,1,0,3,nan,0,nan,1,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,1,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,0,0,1,0,nan,1,DNY02321B31,2018-11-01
3,35,117,nan,0,0,nan,0,0,0,0,3,nan,0,nan,0,0,nan,nan,nan,nan,0,nan,nan,1,nan,nan,0,0,0,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,1,1,1,0,nan,1,DNY01037A11,2018-11-01
4,35,587,nan,0,1,nan,0,0,0,0,3,nan,0,nan,1,0,nan,nan,nan,nan,1,nan,nan,0,nan,nan,0,0,1,nan,nan,0,nan,nan,nan,nan,1,1,nan,0,nan,1,nan,0,0,1,0,nan,1,DNY02131A11,2018-11-01


In [57]:
####### get the days missing for each cell in new cm data -------------------------
#dates_old = sorted(cm_data_old.date.unique())
#dates_old = set(dates_old)
#missing_days = pd.DataFrame(columns={'cell','date'})
#iteration=0
#n=0
#for cell in cm_data.cell_name.unique():
#    cell_dates = cm_data[cm_data.cell_name == cell].date.unique()
#    cell_dates = set(cell_dates)
#    intr = cell_dates.intersection(dates_old)
#    x = dates_old.difference(cell_dates)
#    n+=1
#    if(n %100 == 0):
#        print(n)
#    for val in x:
#        missing_days.loc[iteration] = [val,cell]
#        iteration += 1



In [58]:
cm_data.date = pd.to_datetime(cm_data.date)
cm_data_old.date = pd.to_datetime(cm_data_old.date)

In [59]:
cm_data_old=cm_data_old[cm_data_old.date <= pd.to_datetime('2019-04-30')]

In [60]:
print( 'new cm data has ' + str(cm_data.cell_name.unique().shape[0]) +' cells while the old one has '  +  str(cm_data_old.cell_name_anon.unique().shape[0]))

new cm data has 2867 cells while the old one has 2867


In [61]:
new_entries = pd.DataFrame(cm_data.cell_name.value_counts(),columns = ['cell_name'])['cell_name'].sum()
old_entries = pd.DataFrame(cm_data_old.cell_name_anon.value_counts(),columns = ['cell_name_anon'])['cell_name_anon'].sum()
print( ' new cm data has ' + str(new_entries) + ' entries in total while old_cm_data had ' + str(old_entries))

 new cm data has 435875 entries in total while old_cm_data had 442334


In [62]:
cm_data['unique'] = cm_data.cell_name.astype(str) + cm_data.date.astype(str)
cm_data_old['unique'] = cm_data_old.cell_name_anon.astype(str) + cm_data_old.date.astype(str)


In [63]:
cm_data_old[cm_data_old['unique'].isin(cm_data['unique'])].head()

,est_count_handover,call_count,vsDataEUtranCellFDD__crsGain,feature_UplinkCarrierAggregation,feature_UlTrigInterFreqMob,vsDataEUtranCellFDD__pdcchTargetBlerVolte,feature_QuadAntUlPerfPkg,feature_QuadAntDlPerfPkg4x4,feature_QuadAntDlPerfPkg,feature_VoLTEOptimizedCA,dotw,vsDataEUtranCellFDD__ulSrsEnable,feature_UlSpectrumAnalyzer,vsDataENodeBFunction__dlBbCapacityNet,feature_CrossDUCarrierAggregation,feature_FddTddSameENodeB,vsDataEUtranCellFDD__puschPwrOffset64qam,vsDataENodeBFunction__timePhaseMaxDeviationOtdoa,vsDataENodeBFunction__ulBbCapacityNet,vsDataEUtranCellFDD__cellCapMinCellSubCap,feature_Support12Cells,qciProfile__tReorderingDl,vsDataENodeBFunction__caAwareMfbiIntraCellHo,feature_AdvancedDiffForResourceFairSch,vsDataEUtranCellFDD__outOfCoverageSrTimerPeriodicity,vsDataENodeBFunction__enabledUlTrigMeas,feature_CarrierAggregationFddTdd,feature_FourDlCarrierAggregation,feature_MixedModeRadioLte,vsDataEUtranCellFDD__additionalPlmnList_mcc,vsDataENodeBFunction__releaseInactiveUesInactTime,feature_UlInterferenceReporting,vsDataEUtranCellFDD__pZeroNominalPusch,vsDataEUtranCellFDD__eDrxAllowed,vsDataEUtranCellFDD__threshServingLow,vsDataENodeBFunction__mfbiSupport,feature_SrvccToUtran,feature_DynamicScellSelection,vsDataENodeBFunction__inactivitySupervisionTimerNb,feature_MultiFreqBand,vsDataEUtranCellFDD__cellDownlinkCaCapacity,feature_ThreeDlCarrierAggregation,vsDataEUtranCellFDD__lastSchedLinkAdaptEnabled,feature_RelInactiveUeAtHighLoadHo,feature_PucchOverdimensioning,feature_PdcchPowerBoost,feature_UeThroughputAwareIflb,qciProfile__inactivityTimerOffset,feature_SgwRelocationAtX2Handover,cell_name_anon,date,unique


In [64]:
cm_data.drop_duplicates(inplace=True)

In [65]:
cm_data_old.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 442334 entries, 0 to 446330
Data columns (total 52 columns):
est_count_handover                                      442334 non-null int64
call_count                                              442334 non-null int64
vsDataEUtranCellFDD__crsGain                            0 non-null float64
feature_UplinkCarrierAggregation                        442334 non-null int64
feature_UlTrigInterFreqMob                              442334 non-null int64
vsDataEUtranCellFDD__pdcchTargetBlerVolte               0 non-null float64
feature_QuadAntUlPerfPkg                                442334 non-null int64
feature_QuadAntDlPerfPkg4x4                             442334 non-null int64
feature_QuadAntDlPerfPkg                                442334 non-null int64
feature_VoLTEOptimizedCA                                442334 non-null int64
dotw                                                    442334 non-null int64
vsDataEUtranCellFDD__ulSrsEnable     

In [66]:
cm_data.drop_duplicates(inplace =True)

In [67]:
pd.DataFrame(cm_data_old.cell_name_anon.value_counts(),columns = {'cell_name_anon'})['cell_name_anon'].sum()

442334

In [68]:
#make a dataframe with only one low performance out of 7 days and the previous day
#changes_df = make_changes_df(test_k2_7.copy(),2)
changes_df = test_k2_7.copy()
changes_count = pd.DataFrame(columns = {'cm_changed','cm_not_changed'},index = {'deviation','no_deviation'})


In [69]:
changes_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18162 entries, 0 to 18161
Data columns (total 13 columns):
cell_name_anon          18162 non-null object
band                    18162 non-null object
mean_score_90days       18162 non-null float64
date                    18162 non-null datetime64[ns]
rank                    18162 non-null object
percentile              18162 non-null float64
score_mean_good_days    18162 non-null float64
score_std_90days        18162 non-null float64
score_mean_90days       18162 non-null float64
mu+2sigma               18162 non-null float64
mu-2sigma               18162 non-null float64
gt_mu+2sigma            18162 non-null bool
lt_mu-2sigma            18162 non-null bool
dtypes: bool(2), datetime64[ns](1), float64(7), object(3)
memory usage: 1.7+ MB


In [70]:
pd.DataFrame(cm_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 435875 entries, 0 to 436292
Data columns (total 52 columns):
est_count_handover                                      435875 non-null int64
call_count                                              435875 non-null int64
vsDataEUtranCellFDD__crsGain                            0 non-null float64
feature_UplinkCarrierAggregation                        435875 non-null int64
feature_UlTrigInterFreqMob                              435875 non-null int64
vsDataEUtranCellFDD__pdcchTargetBlerVolte               0 non-null float64
feature_QuadAntUlPerfPkg                                435875 non-null int64
feature_QuadAntDlPerfPkg4x4                             435875 non-null int64
feature_QuadAntDlPerfPkg                                435875 non-null int64
feature_VoLTEOptimizedCA                                435875 non-null int64
dotw                                                    435875 non-null int64
vsDataEUtranCellFDD__ulSrsEnable     

""


In [71]:
cm_data.shape

(435875, 52)

In [72]:
cm_data.dropna(axis=1,inplace=True)
cm_data.head()

,est_count_handover,call_count,feature_UplinkCarrierAggregation,feature_UlTrigInterFreqMob,feature_QuadAntUlPerfPkg,feature_QuadAntDlPerfPkg4x4,feature_QuadAntDlPerfPkg,feature_VoLTEOptimizedCA,dotw,feature_UlSpectrumAnalyzer,feature_CrossDUCarrierAggregation,feature_FddTddSameENodeB,feature_Support12Cells,feature_AdvancedDiffForResourceFairSch,feature_CarrierAggregationFddTdd,feature_FourDlCarrierAggregation,feature_MixedModeRadioLte,feature_UlInterferenceReporting,feature_SrvccToUtran,feature_DynamicScellSelection,feature_MultiFreqBand,feature_ThreeDlCarrierAggregation,feature_RelInactiveUeAtHighLoadHo,feature_PucchOverdimensioning,feature_PdcchPowerBoost,feature_UeThroughputAwareIflb,feature_SgwRelocationAtX2Handover,cell_name,date,unique
0,165,1997,0,1,1,1,1,0,3,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,DNY02321B11,2018-11-01,DNY02321B112018-11-01
1,134,1033,0,1,1,1,1,0,3,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,DNY02321B21,2018-11-01,DNY02321B212018-11-01
2,152,1274,0,1,1,1,1,0,3,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,DNY02321B31,2018-11-01,DNY02321B312018-11-01
3,35,117,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,1,DNY01037A11,2018-11-01,DNY01037A112018-11-01
4,35,587,0,1,0,0,0,0,3,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,DNY02131A11,2018-11-01,DNY02131A112018-11-01


In [75]:
cm_data_changes,cm_data_changes_freq = get_counter_info(cm_data.drop({'call_count','cell_name','date','est_count_handover','dotw'},axis=1).copy())


changes_cm_even.shape :  (217728, 25) 
 changes_cm_odd.shape :  (217728, 25)


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[array([[False, False, False, ..., False, False, False],
       [False,  True,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False]])]
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
n

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


...

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [77]:
cm_data_changes_freq.head()

,feature_UplinkCarrierAggregation,feature_UlTrigInterFreqMob,feature_QuadAntUlPerfPkg,feature_QuadAntDlPerfPkg4x4,feature_QuadAntDlPerfPkg,feature_VoLTEOptimizedCA,feature_UlSpectrumAnalyzer,feature_CrossDUCarrierAggregation,feature_FddTddSameENodeB,feature_Support12Cells,feature_AdvancedDiffForResourceFairSch,feature_CarrierAggregationFddTdd,feature_FourDlCarrierAggregation,feature_MixedModeRadioLte,feature_UlInterferenceReporting,feature_SrvccToUtran,feature_DynamicScellSelection,feature_MultiFreqBand,feature_ThreeDlCarrierAggregation,feature_RelInactiveUeAtHighLoadHo,feature_PucchOverdimensioning,feature_PdcchPowerBoost,feature_UeThroughputAwareIflb,feature_SgwRelocationAtX2Handover,unique
0,"0 , 1","1 , 0","1 , 0","1 , 0","1 , 0","1 , 0","0 , 1","1 , 0","0 , 1","1 , 0","0 , 1","1 , 0","0 , 1","1 , 0","0 , 1","0 , 1","1 , 0","0 , 1","1 , 0","0 , 1","0 , 1","0 , 1","1 , 0","1 , 0","DNY02321B112018-11-01 , DNY02321B212018-11-01"
1,"1 , 0","1 , 0","0 , 1","1 , 0","1 , 0","0 , 1","0 , 1","1 , 0","0 , 1","1 , 0","0 , 1","0 , 1","0 , 1","1 , 0","0 , 1","0 , 1","1 , 0","1 , 0","1 , 0","0 , 1","0 , 1","0 , 1","0 , 1","0 , 1","DNY02321B312018-11-01 , DNY01037A112018-11-01"
2,"1 , 0","1 , 0","1 , 0","1 , 0","1 , 0","1 , 0","1 , 0","1 , 0","1 , 0","1 , 0","0 , 1","0 , 1","1 , 0","1 , 0","1 , 0","1 , 0","1 , 0","0 , 1","1 , 0","0 , 1","1 , 0","0 , 1","0 , 1","0 , 1","DNY02131A112018-11-01 , DNY02131A212018-11-01"
3,"0 , 1","0 , 1","1 , 0","1 , 0","1 , 0","0 , 1","0 , 1","0 , 1","1 , 0","1 , 0","0 , 1","0 , 1","0 , 1","1 , 0","1 , 0","1 , 0","0 , 1","0 , 1","1 , 0","0 , 1","0 , 1","0 , 1","1 , 0","0 , 1","DNY02131A312018-11-01 , DNY01038A112018-11-01"
4,"0 , 1","0 , 1","1 , 0","1 , 0","1 , 0","0 , 1","1 , 0","0 , 1","0 , 1","1 , 0","1 , 0","1 , 0","1 , 0","0 , 1","1 , 0","1 , 0","0 , 1","0 , 1","0 , 1","0 , 1","1 , 0","1 , 0","0 , 1","0 , 1","DNY01034A112018-11-01 , DNY01034A212018-11-01"


In [78]:
# make a dataframe with cm data of only the changes days i.e. deviation df 
cm_data.rename(columns = {'cell_name':'cell_name_anon'},inplace=True)
cm_data.date = pd.to_datetime(cm_data.date)
changes_cm = pd.merge(changes_df[['cell_name_anon','date']],cm_data,on = ['cell_name_anon','date'], how = 'left')
changes_cm['unique'] = changes_cm['cell_name_anon'].astype(str) + changes_cm['date'].astype(str)


In [79]:
# make a df with only the changes of cm data
changes_cm.drop(['call_count','date','cell_name_anon','est_count_handover','dotw'],axis=1,inplace=True)
counter_cm_k2,counter_cm_k2_freq = get_counter_info(changes_cm.copy())


changes_cm_even.shape :  (9081, 25) 
 changes_cm_odd.shape :  (9081, 25)


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[array([[ True,  True,  True, ...,  True,  True,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])]


In [80]:
cm_data.head()

,est_count_handover,call_count,feature_UplinkCarrierAggregation,feature_UlTrigInterFreqMob,feature_QuadAntUlPerfPkg,feature_QuadAntDlPerfPkg4x4,feature_QuadAntDlPerfPkg,feature_VoLTEOptimizedCA,dotw,feature_UlSpectrumAnalyzer,feature_CrossDUCarrierAggregation,feature_FddTddSameENodeB,feature_Support12Cells,feature_AdvancedDiffForResourceFairSch,feature_CarrierAggregationFddTdd,feature_FourDlCarrierAggregation,feature_MixedModeRadioLte,feature_UlInterferenceReporting,feature_SrvccToUtran,feature_DynamicScellSelection,feature_MultiFreqBand,feature_ThreeDlCarrierAggregation,feature_RelInactiveUeAtHighLoadHo,feature_PucchOverdimensioning,feature_PdcchPowerBoost,feature_UeThroughputAwareIflb,feature_SgwRelocationAtX2Handover,cell_name_anon,date,unique
0,165,1997,0,1,1,1,1,0,3,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,DNY02321B11,2018-11-01,DNY02321B112018-11-01
1,134,1033,0,1,1,1,1,0,3,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,DNY02321B21,2018-11-01,DNY02321B212018-11-01
2,152,1274,0,1,1,1,1,0,3,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,DNY02321B31,2018-11-01,DNY02321B312018-11-01
3,35,117,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,1,1,0,1,1,1,1,0,1,DNY01037A11,2018-11-01,DNY01037A112018-11-01
4,35,587,0,1,0,0,0,0,3,0,1,0,1,0,0,0,1,0,1,1,0,1,0,0,1,0,1,DNY02131A11,2018-11-01,DNY02131A112018-11-01


In [81]:
counter_cm_k2.drop('unique',axis = 1, inplace = True)
# count the number of non null rows
count=0
#cm_changes_days = []
for i in counter_cm_k2.index:
    if(any(counter_cm_k2.iloc[i])&('n' not in counter_cm_k2.loc[i,'feature_UplinkCarrierAggregation'])):
        count+=1
        #cm_changes_days.append(i)
changes_count.loc['deviation','cm_changed'] = count
print('got the number of instances where there was deviation & cm also changed : ',count)

got the number of instances where there was deviation & cm also changed :  2131


In [82]:
#----------------get the number of instances where deviation occured but there was no cm change ------------------------------
# make a changes_df
count =  changes_df.shape[0]-count
changes_count.loc['deviation','cm_not_changed'] = count # doesn't include days when there was no cm data

print('got the number of days where deviation occured but there was no cm change, : ',count)


got the number of days where deviation occured but there was no cm change, :  16031


In [88]:
#-----------------get the number of instances when there was no deviation ---------------------------------
# i.e. take test_k2_7 out of test_k2 as considering only 7  days deviation as deviation and everything else as normal behaviour
test_k2['unique'] = test_k2['cell_name_anon'].astype(str) + test_k2['date'].astype(str)
test_k2_7['unique'] = test_k2_7['cell_name_anon'].astype(str) + test_k2_7['date'].astype(str)
test_k2_nodev = test_k2[~test_k2['unique'].isin(test_k2_7.unique)] 
test_k2_nodev.reset_index(inplace=True,drop=True)
# get the df with cm data for these normal days
test_k2_nodev.date= pd.to_datetime(test_k2_nodev.date )
changes_cm_nrml = pd.merge(test_k2_nodev[['cell_name_anon','date','unique']],cm_data,on=['cell_name_anon','date'],how = 'left')
# make a df with only the changes of cm data
changes_cm_nrml.drop(['call_count','date','cell_name_anon','est_count_handover','dotw'],axis=1,inplace=True)
changes_cm_nrml.drop('unique_x',axis=1,inplace=True)
changes_cm_nrml.rename(columns = {'unique_y':'unique'},inplace=True)
counter_nocm_k2,counter_nocm_k2_freq = get_counter_info(changes_cm_nrml.copy())



changes_cm_even.shape :  (206691, 25) 
 changes_cm_odd.shape :  (206691, 25)


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])]


In [89]:
df.cell_name_anon.unique().shape

(9132,)

In [92]:
#counter_nocm_k2.drop('unique',axis = 1, inplace = True)
# count the number of non null rows
count=0
#cm_changes_days = []
for i in counter_nocm_k2.index:
    #print(i)
    if(any(counter_nocm_k2.iloc[i])&('n' not in counter_nocm_k2.loc[i,'feature_UplinkCarrierAggregation'])):
        #print(counter_nocm_k2.iloc[i])
        count+=1
        #cm_changes_days.append(i)
changes_count.loc['no_deviation','cm_changed'] = count
print('got the number of instances where there was deviation & cm also changed : ',count)

got the number of instances where there was deviation & cm also changed :  189367


In [93]:
#----------------get the number of instances where deviation occured but there was no cm change ------------------------------
# make a changes_df
count =  test_k2_nodev.shape[0]-count
changes_count.loc['no_deviation','cm_not_changed'] = count # doesn't include days when there was no cm data

print('got the number of days where deviation occured but there was no cm change, : ',count)


got the number of days where deviation occured but there was no cm change, :  223992


In [94]:
changes_count

,cm_not_changed,cm_changed
deviation,16031,2131
no_deviation,223992,189367


In [95]:
test_k2.shape

(431521, 14)

In [98]:
# get the number of changes of each cm counter for deviation days
feat_chngs_dev = pd.DataFrame()
try :
    counter_cm_freq_k2.drop('unique',inplace=True,axis=1)
except:
    print('no unique to drop')
#col = 'feature_AdvancedDiffForResourceFairSch'
for col in counter_cm_k2_freq.columns:
    x = pd.DataFrame(counter_cm_k2_freq[col].value_counts()).dropna()
    x.replace(' ',np.nan)
    x = x[~(x.index.str.contains('n'))]
    x=x.reset_index()
    x.rename(columns = {'index':'range',col:'count'},inplace=True)
    x.drop(x['count'].idxmax(),axis=0,inplace=True)
    x['feature'] = col
    feat_chngs_dev=feat_chngs_dev.append(x)
#   x

no unique to drop


In [99]:
counter_cm_k2_freq.head()

,feature_UplinkCarrierAggregation,feature_UlTrigInterFreqMob,feature_QuadAntUlPerfPkg,feature_QuadAntDlPerfPkg4x4,feature_QuadAntDlPerfPkg,feature_VoLTEOptimizedCA,feature_UlSpectrumAnalyzer,feature_CrossDUCarrierAggregation,feature_FddTddSameENodeB,feature_Support12Cells,feature_AdvancedDiffForResourceFairSch,feature_CarrierAggregationFddTdd,feature_FourDlCarrierAggregation,feature_MixedModeRadioLte,feature_UlInterferenceReporting,feature_SrvccToUtran,feature_DynamicScellSelection,feature_MultiFreqBand,feature_ThreeDlCarrierAggregation,feature_RelInactiveUeAtHighLoadHo,feature_PucchOverdimensioning,feature_PdcchPowerBoost,feature_UeThroughputAwareIflb,feature_SgwRelocationAtX2Handover,unique
0,"0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","DBQ06063A112019-01-15 , DBQ06063A112019-01-16"
1,"1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","DBQ06063A112019-01-23 , DBQ06063A112019-01-24"
2,"1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","DBQ06063A112019-01-25 , DBQ06063A112019-01-27"
3,"1.0 , 0.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","DBQ06063A112019-01-28 , DBQ06063A112019-01-29"
4,"0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","1.0 , 0.0","1.0 , 0.0","0.0 , 1.0","0.0 , 1.0","1.0 , 0.0","0.0 , 1.0","DBQ06063A112019-01-30 , DBQ06063A112019-01-31"


In [100]:
#getting a dataframe where the total number of time every feature changed value & caused a deviation is noted
feature_change_count_dev = pd.DataFrame(feat_chngs_dev[['feature','count']].groupby('feature').sum()).reset_index().rename(columns = {'index':'feature'})
feature_change_count_dev.head()

,feature,count
0,feature_AdvancedDiffForResourceFairSch,709
1,feature_CarrierAggregationFddTdd,356
2,feature_CrossDUCarrierAggregation,749
3,feature_DynamicScellSelection,264
4,feature_FddTddSameENodeB,228


In [105]:
# get the number of changes of each cm counter for no deviation days
feat_chngs_nodev = pd.DataFrame()
try :
    counter_nocm_freq_k2.drop('unique',inplace=True,axis=1)
except:
    print('no unique to drop')
#col = 'feature_AdvancedDiffForResourceFairSch'
for col in counter_nocm_k2_freq.columns:
    x = pd.DataFrame(counter_nocm_k2_freq[col].value_counts()).dropna()
    x.replace(' ',np.nan)
    x = x[~(x.index.str.contains('n'))]
    x=x.reset_index()
    x.rename(columns = {'index':'range',col:'count'},inplace=True)
    x.drop(x['count'].idxmax(),axis=0,inplace=True)
    x['feature'] = col
    feat_chngs_nodev=feat_chngs_nodev.append(x)
#getting a dataframe where the total number of time every feature changed value & caused a deviation is noted
feature_change_count_nodev = pd.DataFrame(feat_chngs_nodev[['feature','count']].groupby('feature').sum()).reset_index().rename(columns = {'index':'feature'})
feature_change_count_nodev.head()

no unique to drop


,feature,count
0,feature_AdvancedDiffForResourceFairSch,1594
1,feature_CarrierAggregationFddTdd,655
2,feature_CrossDUCarrierAggregation,1745
3,feature_DynamicScellSelection,1528
4,feature_FddTddSameENodeB,484


In [106]:
#makign a dataframe with values of changes_count & no_changes_count is
features_count = pd.merge(feature_change_count_dev,feature_change_count_nodev,on = 'feature',how='inner')
features_count.rename(columns = {'count_x':'deviation_caused','count_y':'no_deviation_caused'},inplace=True)
features_count['ratio'] = features_count['deviation_caused'] / features_count['no_deviation_caused']
features_count.sort_values(by = 'ratio',ascending = False,inplace=True)


In [107]:
#get the sum of all times the values of cm changed without deviatuoin
features_count['no_deviation_caused'].sum()

216328

In [108]:
features_count['ratio']  = features_count['ratio'].round(4)
features_count

,feature,deviation_caused,no_deviation_caused,ratio
21,feature_UlTrigInterFreqMob,446,544,0.82
13,feature_RelInactiveUeAtHighLoadHo,429,538,0.80
20,feature_UlSpectrumAnalyzer,381,534,0.71
5,feature_FourDlCarrierAggregation,482,837,0.58
1,feature_CarrierAggregationFddTdd,356,655,0.54
6,feature_MixedModeRadioLte,710,1426,0.50
9,feature_PucchOverdimensioning,727,1486,0.49
4,feature_FddTddSameENodeB,228,484,0.47
0,feature_AdvancedDiffForResourceFairSch,709,1594,0.44
2,feature_CrossDUCarrierAggregation,749,1745,0.43


In [122]:
features_count.to_csv(r'C:\Users\ezchave\Desktop\work_no\cell ranking\features_ratio_700.csv')

In [118]:
# lets' see the distribution of changes for the highest ratio feature for deviation days
feat_chngs_dev[feat_chngs_dev.feature =='feature_UlTrigInterFreqMob'].head(20)

,range,count,feature
1,"0.0 , 1.0",228,feature_UlTrigInterFreqMob
2,"1.0 , 0.0",218,feature_UlTrigInterFreqMob


In [119]:
# lets' see the distribution of changes for the highest ratio feature for no deviation days
feat_chngs_nodev[feat_chngs_nodev.feature =='feature_UlTrigInterFreqMob'].head(20)

,range,count,feature
1,"0.0 , 1.0",273,feature_UlTrigInterFreqMob
2,"1.0 , 0.0",271,feature_UlTrigInterFreqMob


In [120]:
# lets' see the distribution of changes for the second highest ratio feature for deviation days
feat_chngs_dev[feat_chngs_dev.feature =='feature_RelInactiveUeAtHighLoadHo'].head(20)

,range,count,feature
1,"0.0 , 1.0",220,feature_RelInactiveUeAtHighLoadHo
2,"1.0 , 0.0",209,feature_RelInactiveUeAtHighLoadHo


In [121]:
# lets' see the distribution of second changes for the highest ratio feature for no deviation days
feat_chngs_nodev[feat_chngs_nodev.feature =='feature_RelInactiveUeAtHighLoadHo'].head(20)

,range,count,feature
1,"1.0 , 0.0",278,feature_RelInactiveUeAtHighLoadHo
2,"0.0 , 1.0",260,feature_RelInactiveUeAtHighLoadHo


In [113]:
def get_changed_counters_freq(ch1):
    #making the dataFrame for the changing values of cm counters according to different days
    cm_changed = {}
    try :
        ch1.drop({'call_count','est_count_handover','dotw'},axis=1,inplace=True)
    except:
        print('call_count , est_count_handover , dotw no found to drop')
    # if running second time calcualte ch1 & ch2 again
    for i in ch1.index:
        x=pd.DataFrame(ch1.iloc[i]).replace('',np.nan).dropna()
        x = ' || '.join(list(x.index))
        if(x not in cm_changed ):
            cm_changed[x] = 1
        elif(x in cm_changed) :
            #n = cm_changed[x]
            cm_changed[x]+=1
    del(x)
    try:
        del(cm_changed['vsDataEUtranCellFDD..crsGain || feature_HoOscCtrlUE || feature_AdvancedDiffForResourceFairSch || vsDataENodeBFunction..dlBbCapacityNet || feature_Support18Cells || vsDataEUtranCellFDD..qRxLevMin || feature_MixedModeRadioLte || vsDataENodeBFunction..ulBbCapacityNet || feature_QuadAntUlPerfPkg || vsDataEUtranCellFDD..pdcchTargetBlerVolte || vsDataEUtranCellFDD..lastSchedLinkAdaptEnabled || feature_QuadAntDlPerfPkg4x4 || vsDataEUtranCellFDD..threshServingLow || vsDataEUtranCellFDD..cellSubscriptionCapacity || vsDataEUtranCellFDD..cellCapMinCellSubCap || feature_QuadAntDlPerfPkg || vsDataEUtranCellFDD..ulSrsEnable || feature_PucchOverdimensioning || feature_CrossDUCarrierAggregation || vsDataENodeBFunction..caAwareMfbiIntraCellHo || vsDataEUtranCellFDD..pZeroNominalPusch || vsDataEUtranCellFDD..transmissionMode || vsDataEUtranCellFDD..cellDownlinkCaCapacity || feature_UlTrigInterFreqMob || feature_VoLTEOptimizedCA || vsDataEUtranCellFDD..puschPwrOffset64qam || feature_Support24Cells || feature_SrvccToUtran || feature_CsfbForLimitedDualRadioUe || feature_FourDlCarrierAggregation || feature_FddTddSameENodeB || vsDataEUtranCellFDD..ulConfigurableFrequencyStart || feature_CarrierAggregationFddTdd || qciProfile..tReorderingDl || feature_RelInactiveUeAtHighLoadHo || vsDataENodeBFunction..enabledUlTrigMeas || feature_PdcchPowerBoost || vsDataEUtranCellFDD..qQualMin || vsDataENodeBFunction..mfbiSupport || feature_ConfigurableScellPrio || feature_UlSpectrumAnalyzer || vsDataEUtranCellFDD..tReorderingAutoConfiguration || vsDataENodeBFunction..timeAndPhaseSynchCritical || vsDataEUtranCellFDD..outOfCoverageSrTimerPeriodicity || feature_MultiFreqBand || qciProfile..dlMaxHARQTxQci || unique'])
    except:
        print('couldnt del row  with nan data  columns')
    try:
        del(cm_changed['unique'])
    except:
        print('countd not del unique (date&cellname change col)')
    #watching the highest varying counts of counters respective to the low performace days
    sorted_x = sorted(cm_changed.items(), key=operator.itemgetter(1),reverse=True)
    pd.set_option('display.max_columns',None)
    pd.options.display.max_colwidth = 150
    return pd.DataFrame(sorted_x)

In [114]:
#test_k2.date.unique()

In [115]:
# let's see the combination of features which change most causing a deviation when changed together
sorted_x_dev = get_changed_counters_freq(counter_cm_k2.copy())
sorted_x_dev = sorted_x_dev.replace('|| unique','')
sorted_x_dev.rename(columns = {0:'feature group',1:'count'},inplace=True)
sorted_x_dev.drop(0,axis=0,inplace=True)
sorted_x_dev.head(20)

call_count , est_count_handover , dotw no found to drop
couldnt del row  with nan data  columns
countd not del unique (date&cellname change col)


,feature group,count
1,feature_UplinkCarrierAggregation || feature_UlTrigInterFreqMob || feature_QuadAntUlPerfPkg || feature_QuadAntDlPerfPkg4x4 || feature_QuadAntDlPerf...,909
2,feature_MultiFreqBand,54
3,feature_SrvccToUtran,48
4,feature_DynamicScellSelection,45
5,feature_PdcchPowerBoost,43
6,feature_Support12Cells,40
7,feature_SgwRelocationAtX2Handover,39
8,feature_ThreeDlCarrierAggregation,37
9,feature_AdvancedDiffForResourceFairSch,35
10,feature_PucchOverdimensioning,34


In [116]:
sorted_x_dev

,feature group,count
1,feature_UplinkCarrierAggregation || feature_UlTrigInterFreqMob || feature_QuadAntUlPerfPkg || feature_QuadAntDlPerfPkg4x4 || feature_QuadAntDlPerf...,909
2,feature_MultiFreqBand,54
3,feature_SrvccToUtran,48
4,feature_DynamicScellSelection,45
5,feature_PdcchPowerBoost,43
6,feature_Support12Cells,40
7,feature_SgwRelocationAtX2Handover,39
8,feature_ThreeDlCarrierAggregation,37
9,feature_AdvancedDiffForResourceFairSch,35
10,feature_PucchOverdimensioning,34


In [117]:
# let's see the combination of features which change most but don't cause a deviation when changed together
sorted_x_nodev = get_changed_counters_freq(counter_nocm_k2.copy())
sorted_x_nodev = sorted_x_nodev.replace('|| unique','')
sorted_x_nodev.rename(columns = {0:'feature group',1:'count'},inplace= True)
sorted_x_nodev.head()

call_count , est_count_handover , dotw no found to drop


KeyboardInterrupt: 

In [ ]:

sorted_x_nodev.rename(columns = {0:'feature group',1:'count'},inplace= True)

In [ ]:
#ignore the value unique
sorted_x_nodev.replace('unique','')